# 🐄 Sistema de Estimación de Peso Bovino - Setup ML

> **BLOQUE 0**: Información del proyecto (markdown - solo lectura)

**Proyecto**: Hacienda Gamelera - Bruno Brito Macedo  
**Responsable**: Persona 2 - Setup Infraestructura ML  
**Objetivo**: Preparar datasets y pipeline para entrenamiento de 7 modelos por raza  
**Duración**: 5-6 días  

---

## 📑 Índice de Bloques (Referencia Rápida)

| Bloque | Nombre | Descripción | Requisitos |
|--------|--------|-------------|------------|
| **0** | Información | Markdown introductorio | Ninguno |
| **1** | Clonar Repositorio | Monta Drive y clona desde GitHub (persistente) | Ninguno (requiere internet) |
| **2** | Importar Módulos | Importa módulos internos | Bloque 1 |
| **3** | Ejemplo Modelo | Bloque de prueba (opcional) | Bloque 2 |
| **3.5** | Limpieza Dependencias | Limpia conflictos y reinstala TensorFlow 2.19.0 | Ninguno |
| **4** | Instalar Dependencias Críticas | TensorFlow 2.19.0, NumPy 2.x, MLflow, ml_dtypes | Bloque 3.5 |
| **5** | Instalar Complementos | Albumentations, OpenCV, herramientas ML y verificaciones | Bloque 4 |
| **6** | Imports Generales | Pandas, numpy, tensorflow | Bloque 5 |
| **7** | Configuración Proyecto | Crea carpetas en Drive (Drive ya montado en Bloque 1) | Bloque 6 |
| **7.5** | Configurar Variables CID | Configura rutas CID Dataset | Bloque 7 (opcional) |
| **8** | CID Dataset | Extrae CID Dataset (OPCIONAL) | Bloque 7.5 + archivo comprimido (opcional) |
| **8.5** | Configurar Kaggle.json | Copia kaggle.json desde Drive | Bloque 7 (opcional) |
| **9** | Kaggle Dataset | Descarga dataset Kaggle | Bloque 8.5 + kaggle.json |
| **10** | Google Images | Scraping opcional | Bloque 7 (opcional) |
| **11** | Resumen Datasets | Muestra resumen | Bloques 8-10 |
| **12** | EDA CID Dataset | Análisis exploratorio (OPCIONAL) | Bloque 8 + metadata.csv (opcional) |
| **13** | Visualizaciones EDA | Gráficos interactivos | Bloque 12 |
| **14** | Análisis por Raza | Análisis por raza | Bloque 12 |
| **15** | Pipeline de Datos | Pipeline con augmentation | Bloque 12 |
| **16** | Arquitectura Modelo | Crea modelo EfficientNetB0 | Bloque 15 |
| **17** | Configurar Entrenamiento | Callbacks y MLflow | Bloque 16 |
| **18** | Entrenamiento | Entrena modelo (2-4h) | Bloque 17 + GPU |
| **19** | Evaluación | Evalúa modelo (R², MAE) | Bloque 18 |
| **20** | Exportar TFLite | Exporta a TFLite | Bloque 19 |
| **21** | Resumen Final | Genera resumen completo | Todos los bloques |

---

## 📋 Checklist de Tareas
- [x] Día 1: Setup Google Colab Pro + dependencias
- [ ] Día 2-3: Descargar y organizar datasets críticos
- [ ] Día 4: Análisis exploratorio de datos (EDA)
- [ ] Día 5-6: Preparar pipeline de datos optimizado

## 🎯 Razas Objetivo (7 razas)
1. **Brahman** - Bos indicus robusto
2. **Nelore** - Bos indicus
3. **Angus** - Bos taurus, buena carne
4. **Cebuinas** - Bos indicus general
5. **Criollo** - Adaptado local
6. **Pardo Suizo** - Bos taurus grande
7. **Jersey** - Lechera, menor tamaño


In [ ]:
# ============================================================
# BLOQUE 1: CONFIGURAR RUTA DEL PROYECTO Y CLONAR REPOSITORIO EN DRIVE
# ============================================================
# 📁 Clona el repositorio desde GitHub a Google Drive (persistente entre sesiones)
# 🔗 Repositorio: https://github.com/Angello-27/bovine-weight-estimation.git
# 💾 Se clona en Drive para que persista entre desconexiones del runtime

import sys
import subprocess
import os
from pathlib import Path

# 🔗 URL del repositorio de GitHub
GITHUB_REPO_URL = 'https://github.com/Angello-27/bovine-weight-estimation.git'

# 🔑 Montar Google Drive primero (si no está montado)
print("🔗 Montando Google Drive...")
try:
    from google.colab import drive
    
    # Verificar si Drive ya está montado
    drive_path = Path('/content/drive')
    if not drive_path.exists() or not any(drive_path.iterdir()):
        print("📁 Google Drive no está montado. Montando ahora...")
        print("💡 Se solicitará autorización para acceder a Google Drive")
        drive.mount('/content/drive')
        print("✅ Google Drive montado exitosamente")
    else:
        print("✅ Google Drive ya está montado")
    
    # Usar Drive como ubicación predeterminada (persistente)
    DRIVE_BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
    USE_DRIVE = True
    
except ImportError:
    print("⚠️ No se puede montar Google Drive (no estamos en Colab)")
    print("💡 Usando /content/ como ubicación temporal")
    DRIVE_BASE_DIR = None
    USE_DRIVE = False
except Exception as e:
    print(f"⚠️ Error al montar Google Drive: {e}")
    print("💡 Usando /content/ como ubicación temporal")
    DRIVE_BASE_DIR = None
    USE_DRIVE = False

# ✅ Ruta donde se clonará o está el repositorio
#    - Preferencia 1: Google Drive (persistente entre sesiones)
#    - Preferencia 2: /content/ (temporal, se pierde al desconectar)
if USE_DRIVE and DRIVE_BASE_DIR:
    BASE_DIR = DRIVE_BASE_DIR
    print(f"\n💾 Usando Google Drive: {BASE_DIR} (persistente entre sesiones)")
else:
    BASE_DIR = Path('/content/bovine-weight-estimation')
    print(f"\n⚠️ Usando ubicación temporal: {BASE_DIR} (se pierde al desconectar)")
    print("💡 Para persistir, monta Google Drive primero")

# Ruta del proyecto ML Training
ML_TRAINING_DIR = BASE_DIR / 'ml-training'

# Validamos que la estructura del proyecto exista antes de continuar.
if ML_TRAINING_DIR.exists() and (ML_TRAINING_DIR / 'src').exists():
    print(f"\n✅ Proyecto ya existe en: {ML_TRAINING_DIR}")
    print("📂 Subcarpetas clave detectadas:")
    print(f"   - Código fuente: {ML_TRAINING_DIR / 'src'}")
    print(f"   - Scripts utilitarios: {ML_TRAINING_DIR / 'scripts'}")
    print(f"   - Configuración: {ML_TRAINING_DIR / 'config'}")
    if USE_DRIVE:
        print(f"\n💾 El proyecto persiste entre sesiones porque está en Google Drive")
else:
    print(f"\n📥 Proyecto no encontrado en: {ML_TRAINING_DIR}")
    print(f"🔗 Clonando repositorio desde GitHub: {GITHUB_REPO_URL}")
    if USE_DRIVE:
        print(f"💾 Se clonará en Google Drive para que persista entre sesiones")
    else:
        print(f"⚠️ Se clonará en /content/ (temporal, se perderá al desconectar)")
    
    # Clonar repositorio si no existe
    try:
        # Crear directorio padre si no existe
        BASE_DIR.parent.mkdir(parents=True, exist_ok=True)
        
        # Eliminar directorio si existe pero está vacío o incompleto
        if BASE_DIR.exists() and not (ML_TRAINING_DIR / 'src').exists():
            print(f"⚠️ Directorio existe pero incompleto. Eliminando {BASE_DIR}...")
            import shutil
            shutil.rmtree(BASE_DIR, ignore_errors=True)
        
        # Clonar repositorio
        print(f"📥 Clonando repositorio...")
        result = subprocess.run(
            ['git', 'clone', GITHUB_REPO_URL, str(BASE_DIR)],
            capture_output=True,
            text=True,
            check=True
        )
        
        print(f"✅ Repositorio clonado exitosamente en: {BASE_DIR}")
        print(f"📂 Estructura del proyecto:")
        print(f"   - Código fuente: {ML_TRAINING_DIR / 'src'}")
        print(f"   - Scripts utilitarios: {ML_TRAINING_DIR / 'scripts'}")
        print(f"   - Configuración: {ML_TRAINING_DIR / 'config'}")
        
        if USE_DRIVE:
            print(f"\n💾 El proyecto está en Google Drive y persistirá entre sesiones")
            print(f"💡 No necesitarás volver a clonarlo en futuras sesiones")
        else:
            print(f"\n⚠️ El proyecto está en /content/ y se perderá al desconectar el runtime")
            print(f"💡 Considera montar Google Drive y volver a ejecutar este bloque")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Error al clonar repositorio: {e}")
        print(f"   stdout: {e.stdout}")
        print(f"   stderr: {e.stderr}")
        print("\n💡 Soluciones:")
        print("   1. Verifica que tienes conexión a internet en Colab")
        print("   2. Verifica que el repositorio existe: https://github.com/Angello-27/bovine-weight-estimation")
        print("   3. Si el repositorio es privado, ejecuta el BLOQUE 0.5 primero para configurar el token")
        raise
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        raise

# Añadimos la carpeta src al PYTHONPATH para que todos los módulos internos sean importables.
sys.path.insert(0, str(ML_TRAINING_DIR / 'src'))

# Verificación final
if ML_TRAINING_DIR.exists() and (ML_TRAINING_DIR / 'src').exists():
    print(f"\n✅ Configuración completada correctamente")
    print(f"📁 Directorio base: {BASE_DIR}")
    print(f"📁 ML Training: {ML_TRAINING_DIR}")
    print(f"🐍 PYTHONPATH actualizado: {ML_TRAINING_DIR / 'src'}")
    if USE_DRIVE:
        print(f"💾 Ubicación: Google Drive (persistente entre sesiones)")
    else:
        print(f"⚠️ Ubicación: /content/ (temporal, se pierde al desconectar)")
else:
    raise RuntimeError(
        f"No se pudo configurar el proyecto. Verifica que {ML_TRAINING_DIR} existe y contiene 'src'."
    )


In [ ]:
# ============================================================
# BLOQUE 2: IMPORTAR MÓDULOS DEL PROYECTO
# ============================================================
# ✅ Importa módulos internos del proyecto (requiere BLOQUE 1 exitoso)

# Data Augmentation
from data.augmentation import get_training_transform, get_aggressive_augmentation, get_validation_transform

# Modelos
from models.cnn_architecture import BreedWeightEstimatorCNN, BREED_CONFIGS

# Evaluación
from models.evaluation.metrics import MetricsCalculator, ModelMetrics

# Exportación TFLite
from models.export.tflite_converter import TFLiteExporter

print("✅ Todos los módulos importados correctamente")
print("\n📦 Módulos disponibles:")
print("   - Data augmentation (Albumentations 2.0.8)")
print("   - CNN architectures (MobileNetV2, EfficientNet)")
print("   - Metrics calculator (R², MAE, MAPE)")
print("   - TFLite exporter (optimizado para móvil)")


In [ ]:
# ============================================================
# BLOQUE 3: EJEMPLO - CREAR MODELO PARA UNA RAZA (OPCIONAL)
# ============================================================
# 🎓 Bloque de prueba para verificar que los módulos funcionan
# ⚠️ Puedes omitir este bloque si ya sabes que todo funciona

# Ejemplo 1: Crear modelo para Brahman
model_brahman = BreedWeightEstimatorCNN.build_model(
    breed_name='brahman',
    base_architecture='mobilenetv2'  # Más rápido que EfficientNet
)

print(f"✅ Modelo creado: {model_brahman.name}")
print(f"📊 Parámetros: {model_brahman.count_params():,}")

# Ver arquitectura
print("\n📐 Arquitectura del modelo:")
model_brahman.summary()


---

## 📝 Próximos Pasos

1. **Descargar datasets** (CID, CattleEyeView, etc.)
2. **Preprocesar datos** con nuestros módulos
3. **Entrenar modelo base** genérico
4. **Fine-tuning por raza** (5 razas)
5. **Recolección propia** (Criollo, Pardo Suizo)
6. **Exportar a TFLite** e integrar en app móvil

> Ver `README.md` y `scripts/train_all_breeds.py` para más ejemplos.



## 🚀 Día 1: Setup Google Colab Pro + Dependencias

In [ ]:
# ============================================================
# BLOQUE 3.5: LIMPIEZA DE DEPENDENCIAS CONFLICTIVAS
# ============================================================
# 🧹 Limpia versiones antiguas/conflictivas de TensorFlow y dependencias
# ⚠️ Ejecuta este bloque ANTES del BLOQUE 4
# 💡 Esto evitará advertencias de compatibilidad en pip

import warnings
warnings.filterwarnings('ignore')

print("🧹 INICIANDO LIMPIEZA DE DEPENDENCIAS CONFLICTIVAS...\n")

# Paso 1: Limpiar caché de pip
print("📦 Limpiando caché de pip...")
!pip cache purge
print("   ✅ Caché de pip limpiado\n")

# Paso 2: Desinstalar versiones antiguas de TensorFlow
print("📦 Desinstalando versiones antiguas de TensorFlow...")
!pip uninstall -y -q tensorflow tensorflow-gpu tf-keras 2>/dev/null || true
print("   ✅ TensorFlow antiguo desinstalado\n")

# Paso 3: Desinstalar paquetes problemáticos que dependen de versiones específicas
print("📦 Desinstalando paquetes con dependencias rígidas...")
packages_to_remove = [
    "tensorflow-decision-forests",
    "dopamine-rl", 
    "tensorflow-text",
    "ydf"
]

for package in packages_to_remove:
    print(f"   - Desinstalando {package}...")
    !pip uninstall -y -q {package} 2>/dev/null || true

print("   ✅ Paquetes problemáticos desinstalados\n")

# Paso 4: Reinstalar TensorFlow 2.19 (versión de Colab)
print("📦 Reinstalando TensorFlow 2.19.0 (versión limpia)...")
!pip install -q --force-reinstall --no-cache-dir "tensorflow==2.19.0"
print("   ✅ TensorFlow 2.19.0 reinstalado limpiamente\n")

# Paso 5: Actualizar ml_dtypes y protobuf a versiones compatibles
print("📦 Actualizando ml_dtypes y protobuf...")
!pip install -q --upgrade --no-cache-dir "ml_dtypes>=0.5.0"
!pip install -q --upgrade --no-cache-dir "protobuf>=5.26.1,<6.0"
print("   ✅ ml_dtypes y protobuf actualizados\n")

# Paso 6: Verificar instalación limpia
print("🔍 Verificando instalación limpia...")
import tensorflow as tf
import numpy as np

try:
    import ml_dtypes
    ml_dtypes_version = ml_dtypes.__version__
except:
    ml_dtypes_version = "No disponible"

try:
    import google.protobuf
    protobuf_version = google.protobuf.__version__
except:
    protobuf_version = "No disponible"

print("\n✅ ESTADO DESPUÉS DE LA LIMPIEZA:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}")
print(f"   - ml_dtypes: {ml_dtypes_version}")
print(f"   - Protobuf: {protobuf_version}")

print("\n✅ LIMPIEZA COMPLETADA EXITOSAMENTE")
print("💡 Ahora ejecuta el BLOQUE 4 para instalar las dependencias sin conflictos")
print("💡 Si aún hay advertencias menores, puedes ignorarlas - no afectarán el funcionamiento")


In [ ]:
# ============================================================
# BLOQUE 4: INSTALACIÓN DE DEPENDENCIAS CRÍTICAS (VERSIÓN LIMPIA)
# ============================================================
# 🔧 Instala dependencias críticas después de la limpieza del BLOQUE 3.5
# ⚠️ Ejecuta SOLO después del BLOQUE 3.5
# ✅ Sin conflictos de versiones

import warnings
warnings.filterwarnings('ignore')

print("📦 INSTALANDO DEPENDENCIAS CRÍTICAS (ENTORNO LIMPIO)...\n")

# Verificar que TensorFlow está limpio
import tensorflow as tf
import numpy as np

print(f"🔍 Verificando versiones base:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}\n")

# Paso 1: Instalar tf-keras (evita advertencia de tensorflow-hub)
print("📦 Instalando tf-keras...")
!pip install -q --no-cache-dir "tf-keras>=2.19.0"
print("   ✅ tf-keras instalado\n")

# Paso 2: Arreglar dependencias menores conflictivas
print("📦 Corrigiendo dependencias menores...")
!pip install -q --no-cache-dir "packaging<25"  # Requerido por MLflow
!pip install -q --no-cache-dir "wrapt<2.0.0,>=1.10.10"  # Requerido por aiobotocore
!pip install -q --no-cache-dir "requests==2.32.4"  # Requerido por google-colab
!pip install -q --no-cache-dir "jedi>=0.16"  # Requerido por IPython
print("   ✅ Dependencias menores corregidas\n")

# Paso 3: Instalar MLflow (compatible con NumPy 2.x y Protobuf 5.x)
# MLflow 2.14.1 es compatible con NumPy 2.x a pesar de sus restricciones
print("📦 Instalando MLflow (compatible con NumPy 2.x y Protobuf 5.x)...")
!pip install -q --no-cache-dir "mlflow==2.14.1"
print("   ✅ MLflow instalado\n")

# Paso 4: Instalar DVC (opcional, para versionado de datos)
print("📦 Instalando DVC...")
!pip install -q --no-cache-dir "dvc[gs,s3]==3.51.1"
print("   ✅ DVC instalado\n")

# Paso 5: Actualizar scikit-learn
print("📦 Actualizando scikit-learn...")
!pip install -q --upgrade --no-cache-dir "scikit-learn>=1.6"
print("   ✅ Scikit-learn actualizado\n")

# Configurar mixed precision para GPU
print("🔍 Configurando TensorFlow...")
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("   ✅ Mixed precision (FP16) activado\n")

# Verificar versiones finales
print("=" * 60)
print("✅ DEPENDENCIAS CRÍTICAS INSTALADAS (ENTORNO LIMPIO):")
print("=" * 60)

print(f"\n📦 VERSIONES INSTALADAS:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}")

try:
    import mlflow
    print(f"   - MLflow: {mlflow.__version__} ✅")
    mlflow_ok = True
except Exception as e:
    print(f"   - MLflow: Error - {e} ❌")
    mlflow_ok = False

try:
    import sklearn
    print(f"   - Scikit-learn: {sklearn.__version__}")
except:
    print(f"   - Scikit-learn: No disponible")

try:
    import google.protobuf
    print(f"   - Protobuf: {google.protobuf.__version__}")
except:
    print(f"   - Protobuf: No disponible")

try:
    import ml_dtypes
    print(f"   - ml_dtypes: {ml_dtypes.__version__}")
except:
    print(f"   - ml_dtypes: No disponible")

try:
    from tensorflow import keras
    print(f"   - tf-keras: {keras.__version__}")
except:
    print(f"   - tf-keras: No disponible")

# Verificar compatibilidad
print(f"\n🔍 VERIFICACIÓN DE COMPATIBILIDAD:")
tf_version_ok = tf.__version__.startswith('2.19')
numpy_version_ok = np.__version__.startswith('2.0') or np.__version__.startswith('2.1')

print(f"   - TensorFlow 2.19.x: {'✅' if tf_version_ok else '⚠️'}")
print(f"   - NumPy 2.x: {'✅' if numpy_version_ok else '⚠️'}")
print(f"   - MLflow funcional: {'✅' if mlflow_ok else '❌'}")

if tf_version_ok and numpy_version_ok and mlflow_ok:
    print(f"\n" + "=" * 60)
    print("✅ INSTALACIÓN COMPLETADA EXITOSAMENTE")
    print("=" * 60)
    print(f"\n💡 Continúa con el BLOQUE 5 para instalar complementos")
else:
    print(f"\n⚠️ Hay problemas de compatibilidad. Verifica los errores anteriores.")

print(f"\n📝 NOTAS:")
print(f"   - Entorno limpio sin paquetes conflictivos críticos")
print(f"   - TensorFlow 2.19 + NumPy 2.x + MLflow 2.14.1")
print(f"   - Mixed precision (FP16) configurado para GPU")
print(f"   - Dependencias menores ajustadas")


In [ ]:
# ============================================================
# BLOQUE 5: INSTALACIÓN DE COMPLEMENTOS Y VERIFICACIONES FINALES
# ============================================================
# 🔧 Instala complementos: Albumentations, OpenCV, herramientas ML, etc.

import warnings
warnings.filterwarnings('ignore')

print("📦 INSTALANDO COMPLEMENTOS...\n")

# Paso 1: Instalar Albumentations (compatible con NumPy 2.x)
print("📦 Instalando Albumentations...")
!pip install -q --no-cache-dir "albumentations>=2.0.8"
print("   ✅ Albumentations instalado\n")

# Paso 2: OpenCV ya viene preinstalado en Colab, verificar
print("📦 Verificando OpenCV...")
try:
    import cv2
    print(f"   ✅ OpenCV {cv2.__version__} ya instalado\n")
except:
    print("   - Instalando OpenCV...")
    !pip install -q --no-cache-dir "opencv-python-headless"
    print("   ✅ OpenCV instalado\n")

# Paso 3: Instalar herramientas de ML y datos
print("📦 Instalando herramientas adicionales...")
!pip install -q --no-cache-dir kaggle gdown plotly seaborn
print("   ✅ Herramientas instaladas\n")

# Paso 4: Instalar Pillow actualizado
print("📦 Actualizando Pillow...")
!pip install -q --upgrade --no-cache-dir "pillow>=11.0.0"
print("   ✅ Pillow actualizado\n")

# Verificar versiones instaladas
print("=" * 60)
print("🔍 VERIFICANDO COMPLEMENTOS INSTALADOS:")
print("=" * 60)

import numpy as np
import cv2
import albumentations as A
import sklearn

print(f"\n📦 COMPLEMENTOS:")
print(f"   - NumPy: {np.__version__}")
print(f"   - OpenCV: {cv2.__version__}")
print(f"   - Albumentations: {A.__version__}")
print(f"   - Scikit-learn: {sklearn.__version__}")

# Verificar TensorFlow y GPU
import tensorflow as tf
from tensorflow.keras import mixed_precision

print(f"\n📦 TENSORFLOW:")
print(f"   - TensorFlow: {tf.__version__}")

# Configurar GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"\n✅ GPU DETECTADA Y CONFIGURADA:")
        print(f"   - Dispositivos GPU: {len(gpus)}")
        for i, gpu in enumerate(gpus):
            print(f"   - GPU {i}: {gpu.name}")
        print("\n🎮 GPU lista para entrenamiento")
    except RuntimeError as e:
        print(f"\n⚠️ Error configurando GPU: {e}")
else:
    print(f"\n⚠️ NO SE DETECTÓ GPU")
    print("💡 Activa GPU: Entorno de ejecución > Cambiar tipo > GPU")

print(f"\n✅ Mixed precision (FP16) activado")

print(f"\n" + "=" * 60)
print("✅ TODAS LAS DEPENDENCIAS INSTALADAS CORRECTAMENTE")
print("=" * 60)
print(f"\n💡 Continúa con el BLOQUE 6 (Imports Generales)")
print(f"\n📝 Tu entorno está listo para:")
print(f"   - Entrenar modelos CNN (MobileNetV2, EfficientNet)")
print(f"   - Data augmentation con Albumentations")
print(f"   - Tracking con MLflow")
print(f"   - Exportación a TFLite para móvil")


In [ ]:
# ============================================================
# BLOQUE 6: IMPORTS Y CONFIGURACIÓN GENERAL
# ============================================================
# 🔍 Importa todas las librerías necesarias (pandas, numpy, tensorflow, mlflow, etc.)

# 🔍 Conjunto completo de librerías usadas en el pipeline: utilidades del sistema,
#    ciencia de datos, visualización, ML y tracking de experimentos.
import os
import sys
import shutil
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import json
import requests
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# MLflow para tracking reproducible de experimentos.
# Verificar y corregir versión de MLflow si es necesario

try:
    import mlflow
    import mlflow.tensorflow
    mlflow_ok = True
    mlflow_version = mlflow.__version__
except ImportError as e:
    if '_shut_down_async_logging' in str(e) or 'cannot import name' in str(e):
        print("🔧 Detectado problema con MLflow. Corrigiendo automáticamente...\n")
        
        # Desinstalar MLflow problemático
        print("📦 Desinstalando MLflow problemático...")
        !pip uninstall -y -q mlflow mlflow-skinny 2>/dev/null || true
        print("   ✅ MLflow desinstalado\n")
        
        # Instalar MLflow 2.14.1 (versión estable y probada)
        print("📦 Instalando MLflow 2.14.1 (versión estable)...")
        !pip install -q --no-cache-dir "mlflow==2.14.1"
        print("   ✅ MLflow 2.14.1 instalado\n")
        
        # Intentar importar nuevamente
        try:
            import mlflow
            import mlflow.tensorflow
            mlflow_ok = True
            mlflow_version = mlflow.__version__
            print(f"   ✅ MLflow {mlflow_version} funciona correctamente")
            print(f"   ✅ mlflow.tensorflow importado exitosamente\n")
        except ImportError as e2:
            print(f"   ❌ Error al importar MLflow después de corrección: {e2}")
            print(f"   💡 Intenta reiniciar el runtime: Runtime > Reiniciar sesión\n")
            mlflow_ok = False
            mlflow_version = "Error"
            # Continuar sin MLflow para no bloquear el resto del notebook
            mlflow = None
    else:
        print(f"⚠️ Error al importar MLflow: {e}")
        print(f"💡 MLflow no está disponible, pero puedes continuar sin él\n")
        mlflow_ok = False
        mlflow_version = "Error"
        mlflow = None

if mlflow_ok:
    print(f"✅ MLflow {mlflow_version} configurado correctamente")

# Albumentations y OpenCV (requieren BLOQUE 5 ejecutado)
try:
    import cv2
    import albumentations as A
    cv2_version = cv2.__version__
    albumentations_version = A.__version__
    cv2_available = True
except ImportError as e:
    cv2_version = "No disponible (ejecuta BLOQUE 5)"
    albumentations_version = "No disponible (ejecuta BLOQUE 5)"
    cv2_available = False

# Configurar matplotlib para que todas las gráficas se vean consistentes en Colab.
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("\n✅ Todas las dependencias importadas correctamente")
print(f"\n📊 Versiones:")
print(f"   - TensorFlow: {tf.__version__}")
if 'mlflow_ok' in locals() and mlflow_ok:
    print(f"   - MLflow: {mlflow_version} ✅")
elif 'mlflow_ok' in locals():
    print(f"   - MLflow: {mlflow_version} ⚠️")
else:
    print(f"   - MLflow: No verificado")
if cv2_available:
    print(f"   - OpenCV: {cv2_version}")
    print(f"   - Albumentations: {albumentations_version}")
else:
    print(f"   - OpenCV: {cv2_version} ⚠️ (ejecuta BLOQUE 5)")
    print(f"   - Albumentations: {albumentations_version} ⚠️ (ejecuta BLOQUE 5)")


In [ ]:
# ============================================================
# BLOQUE 7: CONFIGURACIÓN DEL PROYECTO Y ESTRUCTURA DE CARPETAS
# ============================================================
# ⚙️ Crea estructura de carpetas para datos y modelos en Drive
# 📁 Usa el mismo BASE_DIR del BLOQUE 1 (proyecto ya clonado en Drive)
# 💡 Drive ya está montado en el BLOQUE 1, así que solo verificamos

from pathlib import Path

# 🔗 Verificar que Drive está montado (ya debería estar montado en el BLOQUE 1)
drive_path = Path('/content/drive')
if not drive_path.exists() or not any(drive_path.iterdir()):
    print("⚠️ Google Drive no está montado. Montando ahora...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive montado exitosamente")
    except Exception as e:
        print(f"❌ Error al montar Google Drive: {e}")
        raise RuntimeError("Google Drive debe estar montado. Ejecuta el BLOQUE 1 primero.")
else:
    print('✅ Google Drive ya está montado (montado en el BLOQUE 1)')

# 📁 Directorio base dentro de tu Drive (mismo que el BLOQUE 1)
#    El proyecto ya está clonado aquí desde el BLOQUE 1
BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')

# Verificar que el proyecto existe (debería existir desde el BLOQUE 1)
if not BASE_DIR.exists():
    print(f"⚠️ El proyecto no existe en {BASE_DIR}")
    print("💡 Ejecuta el BLOQUE 1 primero para clonar el repositorio en Drive")
    raise RuntimeError(f"El proyecto debe existir en {BASE_DIR}. Ejecuta el BLOQUE 1 primero.")
else:
    print(f"✅ Proyecto encontrado en: {BASE_DIR}")

# 📂 Creamos (si no existen) las carpetas estándar para datos crudos, procesados y modelos.
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
AUGMENTED_DIR = DATA_DIR / 'augmented'
MODELS_DIR = BASE_DIR / 'models'
MLRUNS_DIR = BASE_DIR / 'mlruns'

for dir_path in [DATA_DIR, RAW_DIR, PROCESSED_DIR, AUGMENTED_DIR, MODELS_DIR, MLRUNS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# 📊 Configuración de MLflow (tracking local persistente)
# ------------------------------------------------------------
try:
    import mlflow
    mlflow.set_tracking_uri(f"file://{MLRUNS_DIR}")
    mlflow.set_experiment("bovine-weight-estimation")
    mlflow_available = True
except (ImportError, NameError):
    print("⚠️ MLflow no está disponible. El tracking de experimentos no funcionará.")
    print("💡 Ejecuta el BLOQUE 6 para instalar/corregir MLflow")
    mlflow_available = False

# ------------------------------------------------------------
# ⚙️ Configuración general del entrenamiento (hiperparámetros base)
# ------------------------------------------------------------
CONFIG = {
    'image_size': (224, 224),
    'batch_size': 32,
    'epochs': 100,
    'learning_rate': 0.001,
    'validation_split': 0.2,
    'test_split': 0.1,
    'early_stopping_patience': 10,
    'target_r2': 0.95,
    'max_mae': 5.0,
    'max_inference_time': 3.0
}

# ------------------------------------------------------------
# 🐄 Razas objetivo (Santa Cruz, Chiquitanía y Pampa)
# ------------------------------------------------------------
BREEDS = [
    'brahman', 'nelore', 'angus', 'cebuinas',
    'criollo', 'pardo_suizo', 'guzerat', 'holstein'
]

print("✅ Configuración completada correctamente")
print(f"📁 Directorio base: {BASE_DIR}")
print(f"🎯 Razas objetivo: {len(BREEDS)} razas -> {BREEDS}")
if mlflow_available:
    print(f"📊 MLflow tracking: {MLRUNS_DIR} ✅")
else:
    print(f"📊 MLflow tracking: ⚠️ No disponible (ejecuta BLOQUE 6)")


## 📥 Día 2-3: Descargar y Organizar Datasets Críticos


In [ ]:
# ============================================================
# BLOQUE 7.5: DESCARGAR CID DATASET DESDE GITHUB
# ============================================================
# 📥 Descarga el CID Dataset desde GitHub usando git clone (con git-lfs)
# ⚠️ Ejecuta ANTES del BLOQUE 8 para tener el CID Dataset disponible
# 💡 El CID Dataset se descarga desde: https://github.com/bhuiyanmobasshir94/CID.git
# 📊 Total: ~8GB, 17,899 imágenes con metadata de peso

import os
import subprocess
from pathlib import Path

print("=" * 60)
print("📥 DESCARGANDO CID DATASET DESDE GITHUB")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido (debe venir del BLOQUE 7)
if 'RAW_DIR' not in globals():
    # Intentar definir desde BASE_DIR si existe
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        print("⚠️ RAW_DIR no está definido. Ejecuta el BLOQUE 7 primero.")
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Directorio donde se clonará el CID Dataset
CID_CLONE_DIR = RAW_DIR / 'cid'
CID_REPO_URL = 'https://github.com/bhuiyanmobasshir94/CID.git'

# Verificar si el CID Dataset ya está clonado
if CID_CLONE_DIR.exists() and (CID_CLONE_DIR / 'CID').exists():
    print(f"✅ CID Dataset ya está clonado en: {CID_CLONE_DIR / 'CID'}")
    
    # Verificar si tiene contenido
    cid_content = CID_CLONE_DIR / 'CID'
    if any(cid_content.iterdir()):
        print(f"✅ CID Dataset encontrado y listo para usar")
        
        # Buscar metadata.csv
        metadata_files = list(cid_content.rglob('metadata.csv'))
        if metadata_files:
            CID_METADATA_FILE = str(metadata_files[0])
            print(f"✅ Metadata CSV encontrado: {CID_METADATA_FILE}")
        else:
            CID_METADATA_FILE = str(cid_content / 'metadata.csv')
            print(f"⚠️ Metadata CSV no encontrado, pero el dataset está disponible")
        
        # Configurar variables de entorno
        os.environ['CID_DATASET_PATH'] = str(cid_content)
        os.environ['CID_METADATA_FILE'] = CID_METADATA_FILE
        
        print(f"\n📋 Variables de entorno configuradas:")
        print(f"   CID_DATASET_PATH = {cid_content}")
        print(f"   CID_METADATA_FILE = {CID_METADATA_FILE}")
        print(f"\n💡 Próximos pasos:")
        print(f"   - BLOQUE 8: Configurar Kaggle.json (opcional)")
        print(f"   - BLOQUE 9: Preparar otros datasets (local, Kaggle, scraped)")
        print(f"   - BLOQUE 10: Descargar imágenes desde múltiples fuentes (ideal: 200+ por raza)")
        print(f"   - BLOQUE 12: Analizar CID Dataset (opcional)")
    else:
        print(f"⚠️ Directorio CID existe pero está vacío. Re-clonando...")
        # Eliminar directorio vacío
        import shutil
        shutil.rmtree(CID_CLONE_DIR, ignore_errors=True)
        CID_CLONE_DIR.mkdir(parents=True, exist_ok=True)
else:
    # Crear directorio si no existe
    CID_CLONE_DIR.mkdir(parents=True, exist_ok=True)
    
    print(f"📥 Descargando CID Dataset desde GitHub...")
    print(f"🔗 Repositorio: {CID_REPO_URL}")
    print(f"📁 Destino: {CID_CLONE_DIR}")
    print(f"💾 Tamaño estimado: ~8GB (puede tardar varios minutos)")
    print()
    
    # Instalar git-lfs (requerido para archivos grandes)
    print("📦 Instalando git-lfs...")
    try:
        subprocess.run(['git', 'lfs', 'install'], check=True, capture_output=True)
        print("✅ git-lfs instalado correctamente")
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Error instalando git-lfs: {e}")
        print("💡 Intentando continuar sin git-lfs...")
    except FileNotFoundError:
        print("⚠️ git no está disponible. Instalando...")
        subprocess.run(['apt-get', 'update', '-qq'], check=True)
        subprocess.run(['apt-get', 'install', '-y', 'git', 'git-lfs'], check=True)
        subprocess.run(['git', 'lfs', 'install'], check=True)
        print("✅ git y git-lfs instalados")
    
    # Clonar repositorio CID
    print(f"\n📥 Clonando repositorio CID (esto puede tardar varios minutos)...")
    try:
        result = subprocess.run(
            ['git', 'clone', CID_REPO_URL, str(CID_CLONE_DIR / 'CID')],
            capture_output=True,
            text=True,
            check=True
        )
        print("✅ CID Dataset clonado exitosamente")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error al clonar CID Dataset: {e}")
        print(f"   stdout: {e.stdout}")
        print(f"   stderr: {e.stderr}")
        print("\n💡 Soluciones:")
        print("   1. Verifica que tienes conexión a internet en Colab")
        print("   2. Verifica que el repositorio existe: https://github.com/bhuiyanmobasshir94/CID")
        print("   3. Si el repositorio es privado, necesitarás configurar credenciales")
        print("   4. Puedes continuar con otros datasets (Kaggle, Google Images)")
        raise
    
    # Verificar que se clonó correctamente
    cid_content = CID_CLONE_DIR / 'CID'
    if cid_content.exists() and any(cid_content.iterdir()):
        print(f"✅ CID Dataset clonado correctamente en: {cid_content}")
        
        # Buscar metadata.csv
        metadata_files = list(cid_content.rglob('metadata.csv'))
        if metadata_files:
            CID_METADATA_FILE = str(metadata_files[0])
            print(f"✅ Metadata CSV encontrado: {CID_METADATA_FILE}")
        else:
            CID_METADATA_FILE = str(cid_content / 'metadata.csv')
            print(f"⚠️ Metadata CSV no encontrado en la ubicación esperada")
            print(f"💡 Busca manualmente el archivo metadata.csv en: {cid_content}")
        
        # Contar imágenes (aproximado)
        img_count = len(list(cid_content.rglob('*.jpg'))) + len(list(cid_content.rglob('*.png'))) + len(list(cid_content.rglob('*.jpeg')))
        if img_count > 0:
            print(f"📊 Total imágenes encontradas: {img_count:,}")
        
        # Configurar variables de entorno
        os.environ['CID_DATASET_PATH'] = str(cid_content)
        os.environ['CID_METADATA_FILE'] = CID_METADATA_FILE
        
        print(f"\n📋 Variables de entorno configuradas:")
        print(f"   CID_DATASET_PATH = {cid_content}")
        print(f"   CID_METADATA_FILE = {CID_METADATA_FILE}")
        print(f"\n💡 Próximos pasos:")
        print(f"   - BLOQUE 8: Configurar Kaggle.json (opcional)")
        print(f"   - BLOQUE 9: Preparar otros datasets (local, Kaggle, scraped)")
        print(f"   - BLOQUE 10: Descargar imágenes desde múltiples fuentes (ideal: 200+ por raza)")
        print(f"   - BLOQUE 12: Analizar CID Dataset (opcional)")
    else:
        print(f"⚠️ El repositorio se clonó pero no se encontró contenido")
        print(f"💡 Verifica manualmente: {cid_content}")

print(f"\n{'=' * 60}")
print("✅ BLOQUE 7.5 COMPLETADO")
print(f"{'=' * 60}")

In [ ]:
# ============================================================
# BLOQUE 8: CONFIGURAR KAGGLE.JSON DESDE DRIVE (OPCIONAL)
# ============================================================
# 🔑 Copia kaggle.json desde Google Drive a /root/.kaggle/
# ⚠️ Ejecuta SOLO si tienes kaggle.json en Drive y no lo has configurado aún
# 📁 Ajusta la ruta KAGGLE_JSON_PATH según donde esté tu archivo en Drive

from pathlib import Path
import shutil
import subprocess

# 👉 Ruta del kaggle.json en Google Drive (raíz de MyDrive)
KAGGLE_JSON_PATH = Path('/content/drive/MyDrive/kaggle.json')  # <--- Ruta en la raíz de Drive
# KAGGLE_JSON_PATH = Path('/content/drive/MyDrive/keys/kaggle.json')  # Si está en carpeta keys

if KAGGLE_JSON_PATH.exists():
    kaggle_dir = Path('/root/.kaggle')
    kaggle_dir.mkdir(exist_ok=True)
    
    shutil.copy(KAGGLE_JSON_PATH, kaggle_dir / 'kaggle.json')
    subprocess.run(["chmod", "600", "/root/.kaggle/kaggle.json"], check=True)
    
    print("✅ kaggle.json copiado desde Drive a /root/.kaggle/")
    print("🔑 Credenciales de Kaggle configuradas correctamente")
else:
    print(f"⚠️ No se encontró kaggle.json en: {KAGGLE_JSON_PATH}")
    print("💡 Ajusta KAGGLE_JSON_PATH o sube kaggle.json manualmente antes del BLOQUE 9")

In [ ]:
# ============================================================
# BLOQUE 9: PREPARAR DATASETS DE GANADO BOVINO (SOLUCIÓN URGENTE)
# ============================================================
# 📥 Prepara datasets de ganado bovino para entrenamiento
# 🚨 SOLUCIÓN PARA PRESENTACIÓN: Múltiples opciones para conseguir datos rápidamente

import os
import subprocess
import pandas as pd
import shutil
import requests
from pathlib import Path
from PIL import Image
import io
import json

print("=" * 60)
print("📥 PREPARANDO DATASETS DE GANADO BOVINO")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')


def download_images_from_unsplash(breed: str, count: int = 100, output_dir: Path = None):
    """Descarga imágenes desde Unsplash (gratis, sin API key para uso básico)."""
    if output_dir is None:
        output_dir = RAW_DIR / 'downloaded_images' / breed
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📥 Descargando {count} imágenes de '{breed}' desde Unsplash...")
    
    # Unsplash Source API (gratis, sin autenticación para uso básico)
    # Nota: Para producción, usa la API oficial con key
    search_terms = {
        'brahman': 'brahman cattle',
        'nelore': 'nelore cattle',
        'angus': 'angus cattle',
        'cebuinas': 'zebu cattle',
        'criollo': 'criollo cattle',
        'pardo_suizo': 'brown swiss cattle',
        'jersey': 'jersey cattle'
    }
    
    search_term = search_terms.get(breed, breed + ' cattle')
    
    # URLs de ejemplo de Unsplash (en producción, usar API)
    # Por ahora, creamos estructura y damos instrucciones
    print(f"💡 Para descargar imágenes reales:")
    print(f"   1. Ve a: https://unsplash.com/s/photos/{search_term.replace(' ', '-')}")
    print(f"   2. Descarga {count} imágenes manualmente")
    print(f"   3. Sube a: {output_dir}")
    print(f"   4. O usa el script de descarga automática (requiere API key)")
    
    return output_dir


def create_demo_dataset_with_instructions():
    """Crea estructura de dataset demo con instrucciones detalladas."""
    demo_dir = RAW_DIR / 'demo_dataset'
    demo_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n🎯 Creando estructura de dataset demo...")
    
    breeds = ['brahman', 'nelore', 'angus', 'cebuinas', 'criollo', 'pardo_suizo', 'jersey']
    metadata_rows = []
    
    for breed in breeds:
        breed_dir = demo_dir / breed
        breed_dir.mkdir(parents=True, exist_ok=True)
        
        # Crear archivo de instrucciones
        instructions = f"""INSTRUCCIONES PARA DATASET {breed.upper()}
{'=' * 50}

1. FUENTES DE IMÁGENES GRATIS:
   - Unsplash: https://unsplash.com/s/photos/{breed}-cattle
   - Pexels: https://www.pexels.com/search/{breed}%20cattle/
   - Pixabay: https://pixabay.com/images/search/{breed}%20cattle/
   - Google Images: Busca "{breed} cattle" y descarga manualmente

2. ESTRUCTURA:
   - Nombra imágenes: {breed}_001.jpg, {breed}_002.jpg, etc.
   - Mínimo recomendado: 100 imágenes por raza
   - Ideal para presentación: 150+ imágenes por raza

3. METADATA:
   - Crea metadata.csv con columnas:
     * image_path: ruta relativa (ej: {breed}/{breed}_001.jpg)
     * weight_kg: peso en kilogramos (ej: 450.5)
     * breed: nombre de la raza (ej: {breed})
     * age_category: ternero/vaquillona/toro/vaca

4. PESOS PROMEDIO DE REFERENCIA:
   - Brahman: 400-500 kg
   - Nelore: 380-480 kg
   - Angus: 500-600 kg
   - Cebuinas: 350-450 kg
   - Criollo: 300-400 kg
   - Pardo Suizo: 550-650 kg
   - Jersey: 300-400 kg

5. PARA PRESENTACIÓN ACADÉMICA:
   - Puedes usar pesos sintéticos basados en rangos reales
   - Lo importante es tener imágenes y estructura correcta
   - El modelo se entrenará y funcionará para demostración
"""
        
        with open(breed_dir / 'INSTRUCCIONES.txt', 'w', encoding='utf-8') as f:
            f.write(instructions)
        
        # Agregar filas de ejemplo al metadata
        for i in range(1, 6):  # 5 ejemplos por raza
            metadata_rows.append({
                'image_path': f'{breed}/{breed}_{i:03d}.jpg',
                'weight_kg': 400 + (i * 10),  # Pesos de ejemplo
                'breed': breed,
                'age_category': 'vaca' if i % 2 == 0 else 'toro'
            })
    
    # Crear metadata.csv de ejemplo
    df_metadata = pd.DataFrame(metadata_rows)
    metadata_file = demo_dir / 'metadata_template.csv'
    df_metadata.to_csv(metadata_file, index=False)
    
    print(f"✅ Estructura creada en: {demo_dir}")
    print(f"📋 Template de metadata: {metadata_file}")
    print(f"📁 Carpetas por raza creadas con instrucciones")
    
    return demo_dir


def prepare_local_dataset() -> Path | None:
    """Prepara dataset desde imágenes locales."""
    local_dir = RAW_DIR / 'local_images'
    
    if not local_dir.exists():
        return None
    
    img_files = (
        list(local_dir.rglob('*.jpg')) + 
        list(local_dir.rglob('*.png')) + 
        list(local_dir.rglob('*.jpeg'))
    )
    
    if not img_files:
        return None
    
    print(f"✅ Dataset local: {len(img_files):,} imágenes")
    return local_dir


def prepare_kaggle_dataset_small() -> Path | None:
    """Intenta usar dataset pequeño de Kaggle."""
    output_dir = RAW_DIR / 'kaggle'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    zip_files = list(output_dir.glob('*.zip'))
    if zip_files:
        zip_file = zip_files[0]
        file_size_gb = zip_file.stat().st_size / (1024**3)
        
        if file_size_gb <= 14:
            print(f"✅ ZIP encontrado: {zip_file.name} ({file_size_gb:.2f}GB)")
            # Descomprimir si no está extraído
            if not any(output_dir.glob('**/*.jpg')):
                print(f"📦 Descomprimiendo...")
                subprocess.run(['unzip', '-q', str(zip_file), '-d', str(output_dir)], 
                             check=False)
            return output_dir
    
    return None


def create_combined_dataset():
    """Crea dataset combinado desde todas las fuentes."""
    print("\n" + "=" * 60)
    print("🔗 BUSCANDO DATASETS DISPONIBLES")
    print("=" * 60)
    print()
    
    datasets_found = []
    
    # 1. Dataset local
    local = prepare_local_dataset()
    if local:
        datasets_found.append(('local', local))
    
    # 2. Dataset Kaggle pequeño
    kaggle = prepare_kaggle_dataset_small()
    if kaggle:
        datasets_found.append(('kaggle', kaggle))
    
    # 3. Dataset scrapeado
    scraped_dir = RAW_DIR / 'scraped'
    if scraped_dir.exists():
        imgs = list(scraped_dir.rglob('*.jpg')) + list(scraped_dir.rglob('*.png'))
        if imgs:
            datasets_found.append(('scraped', scraped_dir))
    
    if datasets_found:
        print(f"✅ {len(datasets_found)} fuente(s) encontrada(s):")
        total = 0
        for name, path in datasets_found:
            count = len(list(path.rglob('*.jpg')) + list(path.rglob('*.png')))
            print(f"   - {name}: {count:,} imágenes")
            total += count
        print(f"\n📊 TOTAL: {total:,} imágenes disponibles")
        return True
    
    # Si no hay datasets, crear estructura demo
    print("⚠️ No se encontraron datasets")
    print("\n" + "=" * 60)
    print("🚨 SOLUCIÓN URGENTE PARA PRESENTACIÓN")
    print("=" * 60)
    
    demo_dir = create_demo_dataset_with_instructions()
    
    print(f"\n💡 OPCIONES RÁPIDAS PARA CONSEGUIR IMÁGENES:")
    print(f"\n📥 OPCIÓN 1: Descarga Manual (MÁS RÁPIDO - 30 minutos)")
    print(f"   1. Ve a estos sitios (gratis, sin copyright):")
    print(f"      • Unsplash: https://unsplash.com/s/photos/cattle")
    print(f"      • Pexels: https://www.pexels.com/search/cattle/")
    print(f"      • Pixabay: https://pixabay.com/images/search/cow/")
    print(f"   2. Busca por raza: 'brahman cattle', 'nelore cattle', etc.")
    print(f"   3. Descarga 100-150 imágenes por raza")
    print(f"   4. Organiza en: {RAW_DIR / 'local_images'}")
    print(f"   5. Crea metadata.csv con pesos estimados")
    
    print(f"\n📸 OPCIÓN 2: Fotos Reales (MEJOR CALIDAD)")
    print(f"   1. Toma 100+ fotos del ganado en la Hacienda Gamelera")
    print(f"   2. Anota peso real de báscula para cada foto")
    print(f"   3. Sube a: {RAW_DIR / 'local_images'}")
    
    print(f"\n🤖 OPCIÓN 3: Usar BLOQUE 10 (Scraping Automático)")
    print(f"   1. Ejecuta el BLOQUE 10 para scraping de Google Images")
    print(f"   2. Ajusta límites según necesidad")
    print(f"   3. Verifica y organiza las imágenes descargadas")
    
    print(f"\n📋 ESTRUCTURA MÍNIMA PARA PRESENTACIÓN:")
    print(f"   - Mínimo viable: 100 imágenes por raza (700 total)")
    print(f"   - Recomendado: 150 imágenes por raza (1050 total)")
    print(f"   - Ideal: 200+ imágenes por raza (1400+ total)")
    
    print(f"\n✅ Estructura demo creada en: {demo_dir}")
    print(f"💡 Sigue las instrucciones en cada carpeta de raza")
    
    return False


# Ejecutar
try:
    success = create_combined_dataset()
    
    if success:
        print(f"\n✅ BLOQUE 9 COMPLETADO - Datasets listos")
    else:
        print(f"\n⚠️ BLOQUE 9 COMPLETADO - Estructura demo creada")
        print(f"💡 Sigue las instrucciones para agregar imágenes")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'=' * 60}")

In [ ]:
# ============================================================
# BLOQUE 10: DESCARGAR IMÁGENES DESDE MÚLTIPLES FUENTES
# ============================================================
# 🖼️ Descarga imágenes de ganado bovino desde múltiples fuentes
# 🎯 Objetivo: 200+ imágenes por raza para dataset ideal (1400+ total) ⭐
# 💡 Configurable: Cambia IMAGES_PER_BREED según necesidad
# ⚠️ Respeta términos de uso y evita bloqueos
#
# 📁 ESTRUCTURA DE ALMACENAMIENTO:
#    data/raw/scraped/
#    ├── brahman/
#    │   ├── brahman_001.jpg
#    │   ├── brahman_002.jpg
#    │   └── ...
#    ├── nelore/
#    ├── angus/
#    ├── cebuinas/
#    ├── criollo/
#    ├── pardo_suizo/
#    ├── jersey/
#    └── metadata.csv  (generado automáticamente)
#
# 📊 CLASIFICACIÓN DE DATASETS (7 razas):
#    - Pequeño: < 350 imágenes (< 50 por raza promedio)
#    - Mediano-Chico: 350-699 imágenes (50-99 por raza promedio)
#    - Mediano: 700-1049 imágenes (100-149 por raza promedio) ⭐ MÍNIMO VIABLE
#    - Mediano-Grande: 1050-1399 imágenes (150-199 por raza promedio)
#    - Grande: 1400+ imágenes (200+ por raza promedio) ⭐ IDEAL

import os
import subprocess
import requests
import time
import shutil
from pathlib import Path
from PIL import Image
import io

print("=" * 60)
print("🖼️ DESCARGANDO IMÁGENES DE GANADO BOVINO")
print("=" * 60)
print()
print("⚠️ NOTA IMPORTANTE:")
print("   - Algunos sitios bloquean descargas automáticas (HTTP 403)")
print("   - Esto es normal y el script continuará con otras imágenes")
print("   - Los errores se muestran de forma limitada para no saturar la salida")
print("   - Si se interrumpe, puedes ejecutar nuevamente (continúa donde quedó)")
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Configuración de razas con múltiples términos de búsqueda
BREED_SEARCH_TERMS = {
    'brahman': [
        'brahman cattle',
        'brahman cow',
        'brahman bull',
        'brahman ganado',
        'brahman bovino'
    ],
    'nelore': [
        'nelore cattle',
        'nelore cow',
        'nelore bull',
        'nelore ganado',
        'nelore bovino'
    ],
    'angus': [
        'angus cattle',
        'angus cow',
        'angus bull',
        'angus ganado',
        'angus bovino'
    ],
    'cebuinas': [
        'zebu cattle',
        'cebu cattle',
        'indicus cattle',
        'cebuino ganado'
    ],
    'criollo': [
        'criollo cattle',
        'criollo cow',
        'criollo ganado boliviano',
        'criollo bovino'
    ],
    'pardo_suizo': [
        'brown swiss cattle',
        'pardo suizo cattle',
        'brown swiss cow',
        'pardo suizo ganado'
    ],
    'jersey': [
        'jersey cattle',
        'jersey cow',
        'jersey bull',
        'jersey ganado'
    ]
}

# Objetivo: Configurable - Cambia según necesidad
# - 100 imágenes por raza = 700 total (Dataset Mediano - Mínimo viable)
# - 150 imágenes por raza = 1050 total (Dataset Mediano-Grande - Recomendado)
# - 200 imágenes por raza = 1400 total (Dataset Grande - Ideal) ⭐
IMAGES_PER_BREED = 200  # ⭐ Cambia a 200 para dataset ideal (1400+ imágenes)
IMAGES_PER_SEARCH_TERM = 40  # 40 imágenes por término de búsqueda (para alcanzar 200 por raza)


def scrape_with_bing_downloader(breed: str, search_terms: list, output_dir: Path, limit: int = 200):
    """Scraping usando bing-image-downloader con mejor manejo de errores."""
    breed_dir = output_dir / breed
    breed_dir.mkdir(parents=True, exist_ok=True)
    
    # Verificar imágenes existentes (para continuar si se interrumpió)
    existing_imgs = list(breed_dir.glob('*.jpg')) + list(breed_dir.glob('*.png'))
    already_downloaded = len(existing_imgs)
    
    print(f"\n📥 Descargando imágenes para: {breed.upper()}")
    print(f"📁 Destino: {breed_dir}")
    print(f"🎯 Objetivo: {limit} imágenes")
    
    if already_downloaded > 0:
        print(f"✅ Ya existen {already_downloaded} imágenes (continuando desde aquí)")
        if already_downloaded >= limit:
            print(f"✅ Objetivo alcanzado ({already_downloaded} >= {limit})")
            return already_downloaded
    
    downloaded = already_downloaded
    errors_count = 0
    max_errors_per_term = 10  # Limitar errores mostrados
    
    # Intentar usar bing-image-downloader (más confiable que google-images-download)
    try:
        from bing_image_downloader import downloader
        import sys
        from io import StringIO
        
        # Redirigir stderr para capturar errores sin mostrarlos todos
        original_stderr = sys.stderr
        
        for search_term in search_terms:
            if downloaded >= limit:
                break
            
            remaining = limit - downloaded
            batch_size = min(remaining, IMAGES_PER_SEARCH_TERM)
            
            print(f"   🔍 Buscando: '{search_term}' ({batch_size} imágenes)...", end=' ', flush=True)
            
            try:
                # Capturar errores silenciosamente
                sys.stderr = StringIO()
                
                downloader.download(
                    search_term,
                    limit=batch_size,
                    output_dir=str(breed_dir.parent),
                    adult_filter_off=True,
                    force_replace=False,
                    timeout=10,  # Reducir timeout para evitar esperas largas
                    verbose=False
                )
                
                # Restaurar stderr
                sys.stderr = original_stderr
                
                # Contar imágenes descargadas
                # bing-image-downloader crea carpetas con el nombre del término de búsqueda
                # Buscar en múltiples ubicaciones posibles
                possible_term_dirs = [
                    breed_dir.parent / search_term.replace(' ', '_'),
                    breed_dir.parent / search_term,
                    breed_dir.parent / search_term.replace(' ', '-'),
                ]
                
                term_dir = None
                for possible_dir in possible_term_dirs:
                    if possible_dir.exists() and any(possible_dir.iterdir()):
                        term_dir = possible_dir
                        break
                
                if term_dir and term_dir.exists():
                    # Buscar imágenes en la carpeta temporal
                    imgs = list(term_dir.rglob('*.jpg')) + list(term_dir.rglob('*.png')) + list(term_dir.rglob('*.jpeg'))
                    imgs = [img for img in imgs if img.is_file()]  # Solo archivos, no directorios
                    
                    # Mover a carpeta de raza
                    moved = 0
                    for img in imgs:
                        if downloaded >= limit:
                            break
                        
                        # Obtener el siguiente número disponible
                        next_num = downloaded + 1
                        new_name = breed_dir / f"{breed}_{next_num:03d}{img.suffix}"
                        
                        # Si el archivo ya existe, saltarlo
                        if new_name.exists():
                            continue
                        
                        if img.exists():
                            try:
                                # Copiar en lugar de mover para evitar problemas
                                shutil.copy2(img, new_name)
                                downloaded += 1
                                moved += 1
                            except Exception as e:
                                # Si falla, intentar mover
                                try:
                                    img.rename(new_name)
                                    downloaded += 1
                                    moved += 1
                                except Exception:
                                    continue
                    
                    # Limpiar carpeta temporal después de mover
                    try:
                        if term_dir.exists():
                            shutil.rmtree(term_dir, ignore_errors=True)
                    except Exception:
                        pass
                    
                    print(f"✅ {moved} descargadas")
                else:
                    # Buscar imágenes que puedan haberse descargado en otras ubicaciones
                    all_imgs = list(breed_dir.parent.rglob('*.jpg')) + list(breed_dir.parent.rglob('*.png'))
                    # Filtrar solo las que no están en carpetas de raza
                    temp_imgs = [img for img in all_imgs if breed not in str(img.parent)]
                    
                    if temp_imgs:
                        # Intentar mover las imágenes encontradas
                        moved = 0
                        for img in temp_imgs[:batch_size]:
                            if downloaded >= limit:
                                break
                            next_num = downloaded + 1
                            new_name = breed_dir / f"{breed}_{next_num:03d}{img.suffix}"
                            if not new_name.exists() and img.exists():
                                try:
                                    shutil.copy2(img, new_name)
                                    downloaded += 1
                                    moved += 1
                                except Exception:
                                    continue
                        if moved > 0:
                            print(f"✅ {moved} descargadas (desde ubicación alternativa)")
                        else:
                            print(f"⚠️ 0 descargadas")
                            errors_count += 1
                    else:
                        print(f"⚠️ 0 descargadas")
                        errors_count += 1
                
            except Exception as e:
                sys.stderr = original_stderr
                errors_count += 1
                if errors_count <= max_errors_per_term:
                    print(f"⚠️ Error: {str(e)[:50]}...")
                continue
            
            # Pausa entre búsquedas para evitar bloqueos
            time.sleep(1)  # Reducir pausa a 1 segundo
        
        if errors_count > max_errors_per_term:
            print(f"   ⚠️ ({errors_count - max_errors_per_term} errores adicionales ocultos)")
        
        print(f"   ✅ {breed}: {downloaded} imágenes descargadas (de {limit} objetivo)")
        return downloaded
        
    except ImportError:
        print(f"   ⚠️ bing-image-downloader no instalado")
        print(f"   💡 Instalando...")
        subprocess.run(['pip', 'install', '-q', 'bing-image-downloader'], check=False)
        
        # Reintentar
        try:
            from bing_image_downloader import downloader
            return scrape_with_bing_downloader(breed, search_terms, output_dir, limit)
        except:
            pass
    
    # Si no funciona, dar instrucciones manuales
    print(f"   ⚠️ Scraping automático no disponible o con muchos errores")
    print(f"   💡 INSTRUCCIONES MANUALES:")
    print(f"      1. Ve a: https://www.bing.com/images/search?q={breed}+cattle")
    print(f"      2. Descarga {limit} imágenes manualmente")
    print(f"      3. Sube a: {breed_dir}")
    print(f"      4. Nombra: {breed}_001.jpg, {breed}_002.jpg, etc.")
    
    return downloaded  # Retornar lo que se haya descargado hasta ahora


def scrape_google_images_improved():
    """Scraping mejorado con más imágenes y mejor manejo de errores."""
    print("🖼️ DESCARGANDO IMÁGENES DESDE MÚLTIPLES FUENTES")
    print("=" * 60)
    print()
    print("💡 NOTA: Algunos sitios bloquean descargas automáticas (HTTP 403)")
    print("💡 El script continuará descargando lo que pueda y mostrará un resumen")
    print()
    
    output_dir = RAW_DIR / 'scraped'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Instalar bing-image-downloader (más confiable)
    print("📦 Instalando herramientas de descarga...")
    try:
        subprocess.run(['pip', 'install', '-q', 'bing-image-downloader'], check=False)
        print("✅ bing-image-downloader instalado")
    except:
        print("⚠️ Error instalando bing-image-downloader")
    
    total_downloaded = 0
    results_by_breed = {}
    
    # Descargar para cada raza
    for breed, search_terms in BREED_SEARCH_TERMS.items():
        try:
            downloaded = scrape_with_bing_downloader(
                breed, 
                search_terms, 
                output_dir, 
                limit=IMAGES_PER_BREED
            )
            results_by_breed[breed] = downloaded
            total_downloaded += downloaded
        except KeyboardInterrupt:
            print(f"\n⚠️ Descarga interrumpida por el usuario")
            print(f"💡 Hasta ahora: {total_downloaded:,} imágenes descargadas")
            print(f"💡 Puedes continuar ejecutando este bloque nuevamente")
            break
        except Exception as e:
            print(f"⚠️ Error inesperado con {breed}: {e}")
            results_by_breed[breed] = 0
            continue
        
        # Pausa entre razas para evitar bloqueos
        if breed != list(BREED_SEARCH_TERMS.keys())[-1]:
            print("   ⏸️ Pausa de 3 segundos...")
            time.sleep(3)  # Reducir pausa
    
    # Resumen
    print("\n" + "=" * 60)
    print("📊 RESUMEN DE DESCARGA")
    print("=" * 60)
    print()
    
    for breed, count in results_by_breed.items():
        status = "✅" if count >= 100 else "⚠️" if count >= 50 else "❌"
        print(f"{status} {breed.upper()}: {count} imágenes")
    
    print(f"\n🎯 TOTAL DESCARGADO: {total_downloaded:,} imágenes")
    
    # Verificar si alcanzamos el objetivo
    breeds_with_100 = sum(1 for count in results_by_breed.values() if count >= 100)
    breeds_with_50 = sum(1 for count in results_by_breed.values() if count >= 50)
    
    print(f"\n📈 Análisis:")
    print(f"   - Razas con 50+ imágenes: {breeds_with_50}/{len(BREED_SEARCH_TERMS)}")
    print(f"   - Razas con 100+ imágenes: {breeds_with_100}/{len(BREED_SEARCH_TERMS)}")
    
    # Clasificación de datasets según tamaño total
    # 7 razas × umbrales:
    # - Pequeño: < 350 imágenes (< 50 por raza promedio)
    # - Mediano-Chico: 350-699 imágenes (50-99 por raza promedio)
    # - Mediano: 700-1049 imágenes (100-149 por raza promedio)
    # - Mediano-Grande: 1050-1399 imágenes (150-199 por raza promedio)
    # - Grande: 1400+ imágenes (200+ por raza promedio)
    
    print(f"\n📊 CLASIFICACIÓN DEL DATASET:")
    if total_downloaded >= 1400:  # 200+ por raza
        print(f"   ✅ DATASET GRANDE ({total_downloaded:,} imágenes)")
        print(f"   💡 Excelente para entrenamiento robusto (200+ por raza)")
        print(f"   💡 Ideal para producción y alta precisión")
    elif total_downloaded >= 1050:  # 150+ por raza
        print(f"   ✅ DATASET MEDIANO-GRANDE ({total_downloaded:,} imágenes)")
        print(f"   💡 Muy bueno para entrenamiento (150+ por raza)")
        print(f"   💡 Recomendado para presentación académica")
    elif total_downloaded >= 700:  # 100+ por raza
        print(f"   ✅ DATASET MEDIANO ({total_downloaded:,} imágenes)")
        print(f"   💡 Bueno para entrenamiento (100+ por raza)")
        print(f"   💡 Mínimo viable para resultados confiables")
    elif total_downloaded >= 350:  # 50+ por raza
        print(f"   ⚠️ DATASET MEDIANO-CHICO ({total_downloaded:,} imágenes)")
        print(f"   💡 Aceptable para pruebas (50-99 por raza)")
        print(f"   💡 Recomendado: Descargar más para mejor precisión")
    else:  # < 350 imágenes
        print(f"   ❌ DATASET PEQUEÑO ({total_downloaded:,} imágenes)")
        print(f"   💡 Insuficiente para entrenamiento confiable (< 50 por raza)")
        print(f"   💡 Ejecuta nuevamente o descarga manualmente")
    
    # Crear metadata básico
    create_basic_metadata(output_dir, results_by_breed)
    
    return total_downloaded


def create_basic_metadata(output_dir: Path, results_by_breed: dict):
    """Crea metadata.csv básico con pesos estimados."""
    import pandas as pd
    import random
    
    metadata_rows = []
    
    # Pesos promedio por raza (para metadata sintética)
    breed_weights = {
        'brahman': {'min': 400, 'max': 500, 'avg': 450},
        'nelore': {'min': 380, 'max': 480, 'avg': 430},
        'angus': {'min': 500, 'max': 600, 'avg': 550},
        'cebuinas': {'min': 350, 'max': 450, 'avg': 400},
        'criollo': {'min': 300, 'max': 400, 'avg': 350},
        'pardo_suizo': {'min': 550, 'max': 650, 'avg': 600},
        'jersey': {'min': 300, 'max': 400, 'avg': 350}
    }
    
    random.seed(42)  # Para reproducibilidad
    
    for breed, count in results_by_breed.items():
        if count == 0:
            continue
        
        breed_dir = output_dir / breed
        img_files = sorted(list(breed_dir.glob('*.jpg')) + list(breed_dir.glob('*.png')))
        
        weights = breed_weights.get(breed, {'min': 400, 'max': 500, 'avg': 450})
        
        for i, img_file in enumerate(img_files[:count]):
            # Generar peso sintético basado en rango real
            weight = random.uniform(weights['min'], weights['max'])
            
            metadata_rows.append({
                'image_path': f'{breed}/{img_file.name}',
                'weight_kg': round(weight, 1),
                'breed': breed,
                'age_category': random.choice(['ternero', 'vaquillona', 'toro', 'vaca'])
            })
    
    if metadata_rows:
        df_metadata = pd.DataFrame(metadata_rows)   
        metadata_file = output_dir / 'metadata.csv'
        df_metadata.to_csv(metadata_file, index=False)
        print(f"\n💾 Metadata creado: {metadata_file}")
        print(f"📊 Registros: {len(metadata_rows):,}")
        print(f"💡 NOTA: Pesos son estimados. Reemplaza con datos reales si es posible.")


# Ejecutar descarga
try:
    scraped_images = scrape_google_images_improved()
    
    if scraped_images > 0:
        print(f"\n✅ BLOQUE 10 COMPLETADO")
        print(f"📁 Imágenes en: {RAW_DIR / 'scraped'}")
        print(f"💡 Vuelve a ejecutar el BLOQUE 9 para combinar datasets")
        
        if scraped_images < 700:
            print(f"\n⚠️ Dataset pequeño ({scraped_images:,} imágenes)")
            print(f"💡 RECOMENDACIONES:")
            print(f"   1. Ejecuta el BLOQUE 10 nuevamente (puede descargar más)")
            print(f"   2. O descarga manualmente desde:")
            print(f"      • Unsplash: https://unsplash.com/s/photos/cattle")
            print(f"      • Pexels: https://www.pexels.com/search/cattle/")
            print(f"      • Pixabay: https://pixabay.com/images/search/cow/")
            print(f"   3. Organiza en: {RAW_DIR / 'local_images'}")
    else:
        print(f"\n⚠️ BLOQUE 10 COMPLETADO CON ADVERTENCIAS")
        print(f"💡 No se descargaron imágenes automáticamente")
        print(f"\n🔍 Verificando si hay imágenes descargadas manualmente...")
        
        # Verificar si hay imágenes en las carpetas de razas
        scraped_dir = RAW_DIR / 'scraped'
        total_manual = 0
        if scraped_dir.exists():
            for breed in BREED_SEARCH_TERMS.keys():
                breed_dir = scraped_dir / breed
                if breed_dir.exists():
                    imgs = list(breed_dir.glob('*.jpg')) + list(breed_dir.glob('*.png'))
                    if imgs:
                        print(f"   ✅ {breed}: {len(imgs)} imágenes encontradas")
                        total_manual += len(imgs)
        
        if total_manual > 0:
            print(f"\n✅ Se encontraron {total_manual:,} imágenes descargadas manualmente")
            print(f"💡 Ejecuta el BLOQUE 9 para combinar con otros datasets")
        else:
            print(f"\n💡 SOLUCIONES RECOMENDADAS:")
            print(f"   1. 📥 Descarga manual desde sitios gratuitos:")
            print(f"      • Unsplash: https://unsplash.com/s/photos/cattle")
            print(f"      • Pexels: https://www.pexels.com/search/cattle/")
            print(f"      • Pixabay: https://pixabay.com/images/search/cow/")
            print(f"   2. 📁 Organiza las imágenes en: {RAW_DIR / 'local_images'}")
            print(f"      Estructura: local_images/brahman/, local_images/nelore/, etc.")
            print(f"   3. 📸 Usa fotos reales de la Hacienda Gamelera")
            print(f"   4. 🔄 Ejecuta el BLOQUE 9 para verificar y combinar datasets")
        
except Exception as e:
    print(f"\n❌ Error en BLOQUE 10: {e}")
    import traceback
    traceback.print_exc()
    scraped_images = 0

print(f"\n{'=' * 60}")


In [ ]:
# ============================================================
# BLOQUE 11: RESUMEN DE DATASETS DESCARGADOS
# ============================================================
# 📊 Muestra resumen de todos los datasets disponibles
# ⚠️ Requiere: BLOQUE 8, 9, 10 ejecutados (o al menos uno)

def summarize_datasets(cid_df: pd.DataFrame | None = None) -> pd.DataFrame:
    """Resumen de todos los datasets disponibles (solo datos reales)."""
    print("📊 RESUMEN DE DATASETS")
    print("=" * 50)

    datasets_info = []

    if cid_df is not None:
        datasets_info.append({
            'name': 'CID Dataset',
            'images': len(cid_df),
            'description': 'Computer Vision Research - Cattle Image Database',
            'status': '✅ Disponible',
        })
    else:
        datasets_info.append({
            'name': 'CID Dataset',
            'images': 0,
            'description': 'CID sin metadata cargada',
            'status': '⚠️ Pendiente',
        })

    # Kaggle Dataset (maneja caso cuando aún no existe)
    try:
        kaggle_path = kaggle_dataset_path if 'kaggle_dataset_path' in globals() else None
        kaggle_id = KAGGLE_DATASET_ID if 'KAGGLE_DATASET_ID' in globals() else 'N/A'
        if kaggle_path and Path(kaggle_path).exists():
            kaggle_images = len(list(Path(kaggle_path).glob('**/*.jpg')))
            datasets_info.append({
                'name': 'Kaggle Cattle Weight',
                'images': kaggle_images,
                'description': f'Dataset Kaggle ({kaggle_id})',
                'status': '✅ Disponible' if kaggle_images > 0 else '⚠️ Vacío',
            })
        else:
            datasets_info.append({
                'name': 'Kaggle Cattle Weight',
                'images': 0,
                'description': 'Requiere configuración de API Kaggle',
                'status': '⚠️ Pendiente',
            })
    except NameError:
        datasets_info.append({
            'name': 'Kaggle Cattle Weight',
            'images': 0,
            'description': 'Requiere ejecutar BLOQUE 9',
            'status': '⚠️ Pendiente',
        })

    # Google Images Scraped (maneja caso cuando aún no existe)
    try:
        scraped_count = scraped_images if 'scraped_images' in globals() else 0
        datasets_info.append({
            'name': 'Google Images Scraped',
            'images': scraped_count,
            'description': 'Razas locales bolivianas',
            'status': '✅ Disponible' if scraped_count > 0 else '⚠️ Pendiente',
        })
    except NameError:
        datasets_info.append({
            'name': 'Google Images Scraped',
            'images': 0,
            'description': 'Requiere ejecutar BLOQUE 10 (opcional)',
            'status': '⚠️ Pendiente',
        })

    df_datasets = pd.DataFrame(datasets_info)
    print(df_datasets.to_string(index=False))

    total_images = int(df_datasets['images'].sum())
    print(f"\n🎯 TOTAL IMÁGENES DISPONIBLES: {total_images:,}")

    summary_path = DATA_DIR / 'datasets_summary.csv'
    df_datasets.to_csv(summary_path, index=False)
    print(f"\n💾 Resumen guardado en: {summary_path}")

    return df_datasets

# Ejecutar resumen (maneja caso cuando df_cid aún no existe)
# ⚠️ Nota: Si ejecutas ANTES del BLOQUE 12, df_cid será None y solo mostrará datasets de Kaggle/Google
try:
    # Verificar si df_cid existe en el scope global
    df_cid_temp = df_cid if 'df_cid' in globals() else None
    datasets_summary = summarize_datasets(df_cid_temp)
except NameError:
    # Si df_cid no existe aún, ejecutar resumen sin metadata del CID
    print("ℹ️ df_cid aún no cargado. Ejecuta BLOQUE 12 para cargar metadata del CID Dataset.")
    datasets_summary = summarize_datasets(None)


## 📊 Día 4: Análisis Exploratorio de Datos (EDA)


In [ ]:
# ============================================================
# BLOQUE 12: ANÁLISIS EXPLORATORIO - CID DATASET (OPCIONAL)
# ============================================================
# 📊 Carga y analiza metadata del CID Dataset si está disponible
# ⚠️ OPCIONAL: Si no tienes CID Dataset, puedes continuar con otros datasets
# 💡 Este bloque analiza el CID si existe, pero no falla si no está disponible

import os
import json
import pandas as pd
import numpy as np
from pathlib import Path

# Verificar que RAW_DIR está definido (debe venir del BLOQUE 7)
if 'RAW_DIR' not in globals():
    # Intentar definir desde BASE_DIR si existe
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        print("⚠️ RAW_DIR no está definido. Ejecuta el BLOQUE 7 primero.")
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Verificar que BREEDS está definido (debe venir del BLOQUE 7)
if 'BREEDS' not in globals():
    BREEDS = ['brahman', 'nelore', 'angus', 'cebuinas', 'criollo', 'pardo_suizo', 'guzerat', 'holstein']

print("=" * 60)
print("📊 ANÁLISIS EXPLORATORIO - CID DATASET (OPCIONAL)")
print("=" * 60)
print()

# Verificar si CID Dataset está disponible
CID_PATH = None

# 1. Buscar en variable de entorno (configurada por BLOQUE 7.5)
if 'CID_DATASET_PATH' in os.environ:
    env_path = Path(os.environ['CID_DATASET_PATH'])
    if env_path.exists() and any(env_path.iterdir()):
        CID_PATH = env_path
        print(f"✅ CID Dataset encontrado desde variable de entorno: {CID_PATH}")

# 2. Buscar en variables globales
if CID_PATH is None and 'cid_dataset_path' in globals() and cid_dataset_path:
    CID_PATH = Path(cid_dataset_path)

# 3. Buscar en rutas estándar (si BLOQUE 7.5 se ejecutó)
if CID_PATH is None and RAW_DIR.exists():
    possible_cid_paths = [
        RAW_DIR / 'cid' / 'CID',  # Ruta estándar del BLOQUE 7.5
        RAW_DIR / 'cid',
        RAW_DIR / 'cid_dataset' / 'CID',
        RAW_DIR / 'cid_dataset',
    ]
    for path in possible_cid_paths:
        if path.exists() and any(path.iterdir()):
            CID_PATH = path
            break

df_cid = None

if CID_PATH and CID_PATH.exists():
    print(f"🔍 Explorando estructura de CID en: {CID_PATH}")
    
    # Buscar archivo metadata
    metadata_files = list(CID_PATH.rglob('*.csv'))
    
    if metadata_files:
        # Cargar primer CSV encontrado
        metadata_path = metadata_files[0]
        print(f"📄 Cargando metadata: {metadata_path.name}")
        
        try:
            df_cid = pd.read_csv(metadata_path)
            
            print(f"\n📊 Dimensiones: {df_cid.shape[0]} filas × {df_cid.shape[1]} columnas")
            print(f"\n📋 Columnas disponibles:")
            for col in df_cid.columns:
                print(f"  - {col}")
            
            # Mostrar primeras filas
            print(f"\n👀 Primeras 3 filas:")
            print(df_cid.head(3))
            
            # Identificar columna de peso (buscar variaciones)
            weight_col = None
            for col in df_cid.columns:
                if any(keyword in col.lower() for keyword in ['weight', 'peso', 'kg', 'kilogram']):
                    weight_col = col
                    break
            
            if weight_col:
                print(f"\n✅ Columna de peso encontrada: '{weight_col}'")
                
                # Estadísticas de peso
                print(f"\n📊 Estadísticas de Peso:")
                print(df_cid[weight_col].describe())
            
            # Identificar columna de raza
            breed_col = None
            for col in df_cid.columns:
                if any(keyword in col.lower() for keyword in ['breed', 'raza', 'race', 'type']):
                    breed_col = col
                    break
            
            if breed_col:
                print(f"\n✅ Columna de raza encontrada: '{breed_col}'")
                
                # Distribución por raza
                print(f"\n📊 Distribución por Raza:")
                breed_counts = df_cid[breed_col].value_counts()
                print(breed_counts.head(10))
                
                # Mapear a razas objetivo
                print(f"\n🎯 Coincidencias con razas objetivo:")
                for target_breed in BREEDS:
                    matches = df_cid[breed_col].str.contains(target_breed, case=False, na=False).sum()
                    status = "✅" if matches > 50 else "⚠️" if matches > 10 else "❌"
                    print(f"{status} {target_breed.capitalize()}: {matches} imágenes")
            
            print(f"\n✅ CID Dataset analizado exitosamente")
            
        except Exception as e:
            print(f"❌ Error cargando metadata: {e}")
            print("💡 Continuando sin CID Dataset...")
            df_cid = None
    else:
        print("⚠️ No se encontró archivo metadata CSV")
        print("💡 CID podría tener estructura diferente")
        
        # Contar imágenes por subcarpeta
        subdirs = [d for d in CID_PATH.iterdir() if d.is_dir()]
        if subdirs:
            print(f"\n📁 Subdirectorios encontrados ({len(subdirs)}):")
            for subdir in subdirs[:10]:
                img_count = len(list(subdir.glob('*.jpg'))) + len(list(subdir.glob('*.png')))
                print(f"   📂 {subdir.name}: {img_count} imágenes")
        else:
            # Contar imágenes totales
            img_count = len(list(CID_PATH.rglob('*.jpg'))) + len(list(CID_PATH.rglob('*.png')))
            print(f"\n📊 Total de imágenes en CID: {img_count:,}")
else:
    print("⚠️ CID Dataset no disponible (opcional)")
    print("💡 Para usar CID Dataset:")
    print("   1. Ejecuta el BLOQUE 7.5 para descargar el CID Dataset desde GitHub")
    print("   2. El BLOQUE 7.5 descarga automáticamente desde: https://github.com/bhuiyanmobasshir94/CID.git")
    print("   3. Después de ejecutar BLOQUE 7.5, vuelve a ejecutar este BLOQUE 12")

print(f"\n{'=' * 60}")
if df_cid is not None:
    print(f"✅ CID Dataset disponible: {len(df_cid):,} registros")
    print(f"💡 Continúa con el BLOQUE 13 para visualizaciones EDA")
else:
    print(f"⚠️ CID Dataset no disponible (opcional)")
    print(f"💡 Continúa con otros datasets:")
    print(f"   - BLOQUE 9: Descargar dataset de Kaggle")
    print(f"   - BLOQUE 10: Scraping de Google Images (opcional)")
    print(f"   - BLOQUE 11: Ver resumen de datasets disponibles")
    print(f"   - O usa el código de EDA flexible mostrado en el proyecto")
print(f"{'=' * 60}")


In [ ]:
# ============================================================
# BLOQUE 13: VISUALIZACIONES EDA
# ============================================================
# 📊 Crea gráficos interactivos del análisis exploratorio
# ⚠️ Funciona con CID Dataset (BLOQUE 12) o dataset scrapeado (BLOQUE 10)

import pandas as pd
import numpy as np
from pathlib import Path

print("=" * 60)
print("📊 VISUALIZACIONES EDA")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Intentar cargar dataset disponible
df_eda = None
dataset_name = "Desconocido"

# 1. Intentar usar df_cid si está disponible (BLOQUE 12)
if 'df_cid' in globals() and df_cid is not None:
    df_eda = df_cid.copy()
    dataset_name = "CID Dataset"
    print(f"✅ Usando CID Dataset: {len(df_eda):,} registros")
else:
    # 2. Intentar cargar metadata del dataset scrapeado (BLOQUE 10)
    scraped_metadata = RAW_DIR / 'scraped' / 'metadata.csv'
    if scraped_metadata.exists():
        try:
            df_eda = pd.read_csv(scraped_metadata)
            dataset_name = "Dataset Scrapeado"
            print(f"✅ Usando Dataset Scrapeado: {len(df_eda):,} registros")
        except Exception as e:
            print(f"⚠️ Error cargando metadata scrapeado: {e}")
    else:
        print("⚠️ No se encontró ningún dataset disponible")
        print("💡 Ejecuta el BLOQUE 10 para descargar imágenes o el BLOQUE 12 para CID Dataset")

if df_eda is None or len(df_eda) == 0:
    print("\n❌ No hay datos disponibles para visualización")
    print("💡 Ejecuta el BLOQUE 10 para descargar imágenes")
else:
    print(f"\n📋 Columnas disponibles: {list(df_eda.columns)}")
    
    # Verificar columnas requeridas
    required_cols = ['weight_kg', 'breed']
    missing_cols = [col for col in required_cols if col not in df_eda.columns]
    
    if missing_cols:
        print(f"❌ Faltan columnas requeridas: {missing_cols}")
        print("💡 El metadata debe tener al menos: weight_kg, breed")
    else:
        def create_eda_visualizations(df, dataset_name="Dataset"):
            """Crear visualizaciones completas del EDA (adaptado a columnas disponibles)"""
            print(f"\n📊 Creando visualizaciones EDA para {dataset_name}...")
            
            # Determinar qué visualizaciones podemos hacer según columnas disponibles
            has_age = 'age_category' in df.columns
            has_quality = 'image_quality' in df.columns
            has_lighting = 'lighting' in df.columns
            has_angle = 'angle' in df.columns
            
            # Configurar subplots (2x2 es suficiente)
            subplot_titles = ['Distribución de Peso', 'Peso por Raza']
            if has_age:
                subplot_titles.append('Distribución por Edad')
            else:
                subplot_titles.append('Imágenes por Raza')
            
            if has_quality:
                subplot_titles.append('Calidad de Imágenes')
            elif has_lighting:
                subplot_titles.append('Peso vs Iluminación')
            elif has_angle:
                subplot_titles.append('Peso vs Ángulo')
            else:
                subplot_titles.append('Peso Promedio por Raza')
            
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=subplot_titles[:4],
                specs=[[{"secondary_y": False}, {"secondary_y": False}],
                       [{"secondary_y": False}, {"secondary_y": False}]]
            )
            
            # 1. Distribución de peso
            fig.add_trace(
                go.Histogram(x=df['weight_kg'], nbinsx=50, name='Peso (kg)',
                            marker_color='lightblue', opacity=0.7),
                row=1, col=1
            )
            
            # 2. Peso por raza
            for breed in df['breed'].unique()[:10]:  # Limitar a 10 razas para legibilidad
                breed_data = df[df['breed'] == breed]['weight_kg']
                if len(breed_data) > 0:
                    fig.add_trace(
                        go.Box(y=breed_data, name=breed, boxpoints='outliers'),
                        row=1, col=2
                    )
            
            # 3. Distribución por edad o por raza
            if has_age:
                age_counts = df['age_category'].value_counts()
                fig.add_trace(
                    go.Bar(x=age_counts.index, y=age_counts.values, name='Categorías de Edad',
                           marker_color='lightgreen'),
                    row=2, col=1
                )
            else:
                # Si no hay edad, mostrar distribución por raza
                breed_counts = df['breed'].value_counts()
                fig.add_trace(
                    go.Bar(x=breed_counts.index, y=breed_counts.values, name='Imágenes por Raza',
                           marker_color='lightcoral'),
                    row=2, col=1
                )
            
            # 4. Calidad, Iluminación, Ángulo o Peso Promedio
            if has_quality:
                quality_counts = df['image_quality'].value_counts()
                fig.add_trace(
                    go.Pie(labels=quality_counts.index, values=quality_counts.values,
                           name='Calidad'),
                    row=2, col=2
                )
            elif has_lighting:
                for lighting in df['lighting'].unique()[:5]:  # Limitar a 5
                    lighting_data = df[df['lighting'] == lighting]['weight_kg']
                    if len(lighting_data) > 0:
                        fig.add_trace(
                            go.Box(y=lighting_data, name=lighting),
                            row=2, col=2
                        )
            elif has_angle:
                for angle in df['angle'].unique()[:5]:  # Limitar a 5
                    angle_data = df[df['angle'] == angle]['weight_kg']
                    if len(angle_data) > 0:
                        fig.add_trace(
                            go.Box(y=angle_data, name=angle),
                            row=2, col=2
                        )
            else:
                # Si no hay ninguna, mostrar estadísticas de peso por raza
                breed_weights = df.groupby('breed')['weight_kg'].mean().sort_values(ascending=False)
                fig.add_trace(
                    go.Bar(x=breed_weights.index, y=breed_weights.values, 
                           name='Peso Promedio por Raza', marker_color='lightseagreen'),
                    row=2, col=2
                )
            
            # Configurar layout
            fig.update_layout(
                height=1000,
                title_text=f"Análisis Exploratorio - {dataset_name}",
                title_x=0.5,
                showlegend=True
            )
            
            # Mostrar gráfico
            fig.show()
            
            # Guardar gráfico
            if 'DATA_DIR' in globals():
                output_path = DATA_DIR / 'eda_visualizations.html'
            else:
                output_path = RAW_DIR.parent / 'processed' / 'eda_visualizations.html'
                output_path.parent.mkdir(parents=True, exist_ok=True)
            
            fig.write_html(str(output_path))
            print(f"💾 Visualizaciones guardadas en: {output_path}")
            
            return fig
        
        # Ejecutar visualizaciones
        try:
            eda_fig = create_eda_visualizations(df_eda, dataset_name)
            print(f"\n✅ Visualizaciones EDA completadas exitosamente")
        except Exception as e:
            print(f"\n❌ Error creando visualizaciones: {e}")
            import traceback
            traceback.print_exc()

print(f"\n{'=' * 60}")


In [ ]:
# ============================================================
# BLOQUE 14: ANÁLISIS ESPECÍFICO POR RAZA
# ============================================================
# 🐄 Analiza qué razas tienen suficientes datos para entrenamiento
# ⚠️ Funciona con CID Dataset (BLOQUE 12) o dataset scrapeado (BLOQUE 10)

import pandas as pd
import numpy as np
from pathlib import Path

print("=" * 60)
print("🐄 ANÁLISIS POR RAZA PARA ENTRENAMIENTO")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Intentar cargar dataset disponible
df_breed_analysis = None
dataset_name = "Desconocido"

# 1. Intentar usar df_cid si está disponible (BLOQUE 12)
if 'df_cid' in globals() and df_cid is not None:
    df_breed_analysis = df_cid.copy()
    dataset_name = "CID Dataset"
    print(f"✅ Usando CID Dataset: {len(df_breed_analysis):,} registros")
else:
    # 2. Intentar cargar metadata del dataset scrapeado (BLOQUE 10)
    scraped_metadata = RAW_DIR / 'scraped' / 'metadata.csv'
    if scraped_metadata.exists():
        try:
            df_breed_analysis = pd.read_csv(scraped_metadata)
            dataset_name = "Dataset Scrapeado"
            print(f"✅ Usando Dataset Scrapeado: {len(df_breed_analysis):,} registros")
        except Exception as e:
            print(f"⚠️ Error cargando metadata scrapeado: {e}")
    else:
        print("⚠️ No se encontró ningún dataset disponible")
        print("💡 Ejecuta el BLOQUE 10 para descargar imágenes o el BLOQUE 12 para CID Dataset")

if df_breed_analysis is None or len(df_breed_analysis) == 0:
    print("\n❌ No hay datos disponibles para análisis")
    print("💡 Ejecuta el BLOQUE 10 para descargar imágenes")
else:
    # Verificar columnas requeridas
    if 'breed' not in df_breed_analysis.columns or 'weight_kg' not in df_breed_analysis.columns:
        print("❌ El dataset debe tener columnas 'breed' y 'weight_kg'")
        print(f"💡 Columnas disponibles: {list(df_breed_analysis.columns)}")
    else:
        def analyze_breeds_for_training(df):
            """Analizar qué razas están bien representadas para entrenamiento"""
            print(f"\n📊 Analizando razas en {dataset_name}...")
            print("=" * 50)
            
            # Razas objetivo del proyecto
            target_breeds = ['brahman', 'nelore', 'angus', 'cebuinas', 'criollo', 'pardo_suizo', 'jersey']
            
            breed_analysis = []
            
            for breed in target_breeds:
                # Buscar la raza en el dataset (case-insensitive)
                breed_mask = df['breed'].str.lower() == breed.lower()
                breed_data = df[breed_mask]
                
                if len(breed_data) > 0:
                    count = len(breed_data)
                    avg_weight = breed_data['weight_kg'].mean()
                    std_weight = breed_data['weight_kg'].std()
                    
                    # Ajustar umbrales para dataset más pequeño (100+ es suficiente)
                    status = "✅ Suficiente" if count >= 200 else "✅ Bueno" if count >= 100 else "⚠️ Limitado" if count >= 50 else "❌ Insuficiente"
                    
                    strategy = 'Direct training' if count >= 200 else 'Transfer learning' if count >= 100 else 'Data augmentation' if count >= 50 else 'Data collection'
                else:
                    # Si no se encuentra la raza exacta, buscar variaciones
                    breed_variations = [breed, breed.replace('_', ' '), breed.capitalize()]
                    count = 0
                    avg_weight = 0
                    std_weight = 0
                    
                    for variation in breed_variations:
                        mask = df['breed'].str.lower().str.contains(variation.lower(), na=False)
                        if mask.sum() > 0:
                            breed_data = df[mask]
                            count = len(breed_data)
                            avg_weight = breed_data['weight_kg'].mean()
                            std_weight = breed_data['weight_kg'].std()
                            break
                    
                    status = "✅ Bueno" if count >= 100 else "⚠️ Limitado" if count >= 50 else "❌ No encontrado"
                    strategy = 'Transfer learning' if count >= 100 else 'Data augmentation' if count >= 50 else 'Data collection'
                
                breed_analysis.append({
                    'breed': breed,
                    'images_available': count,
                    'avg_weight_kg': round(avg_weight, 1) if avg_weight > 0 else 0,
                    'std_weight_kg': round(std_weight, 1) if std_weight > 0 else 0,
                    'status': status,
                    'strategy': strategy
                })
            
            # Crear DataFrame
            df_result = pd.DataFrame(breed_analysis)
            
            # Mostrar tabla
            print("\n📊 ANÁLISIS POR RAZA:")
            print(df_result.to_string(index=False))
            
            # Guardar análisis
            if 'DATA_DIR' in globals():
                output_path = DATA_DIR / 'breed_analysis.csv'
            else:
                output_path = RAW_DIR.parent / 'processed' / 'breed_analysis.csv'
                output_path.parent.mkdir(parents=True, exist_ok=True)
            
            df_result.to_csv(output_path, index=False)
            print(f"\n💾 Análisis por raza guardado en: {output_path}")
            
            # Recomendaciones adaptadas a dataset más pequeño
            print(f"\n🎯 RECOMENDACIONES PARA ENTRENAMIENTO:")
            
            # Para dataset de 1,269 imágenes, ajustar umbrales
            sufficient_breeds = df_result[df_result['images_available'] >= 200]
            if len(sufficient_breeds) > 0:
                print(f"✅ Entrenamiento directo (200+ imágenes): {', '.join(sufficient_breeds['breed'].tolist())}")
            
            good_breeds = df_result[(df_result['images_available'] >= 100) & (df_result['images_available'] < 200)]
            if len(good_breeds) > 0:
                print(f"✅ Transfer learning recomendado (100-199 imágenes): {', '.join(good_breeds['breed'].tolist())}")
            
            limited_breeds = df_result[(df_result['images_available'] >= 50) & (df_result['images_available'] < 100)]
            if len(limited_breeds) > 0:
                print(f"⚠️ Data augmentation necesario (50-99 imágenes): {', '.join(limited_breeds['breed'].tolist())}")
            
            insufficient_breeds = df_result[df_result['images_available'] < 50]
            if len(insufficient_breeds) > 0:
                print(f"❌ Recolección requerida (< 50 imágenes): {', '.join(insufficient_breeds['breed'].tolist())}")
            
            # Resumen general
            total_images = df_result['images_available'].sum()
            print(f"\n📈 RESUMEN GENERAL:")
            print(f"   - Total imágenes: {total_images:,}")
            print(f"   - Razas con 100+ imágenes: {len(df_result[df_result['images_available'] >= 100])}/{len(target_breeds)}")
            print(f"   - Razas con 150+ imágenes: {len(df_result[df_result['images_available'] >= 150])}/{len(target_breeds)}")
            
            if total_images >= 700:
                print(f"\n✅ Dataset suficiente para entrenamiento ({total_images:,} imágenes)")
                print(f"💡 Puedes continuar con el BLOQUE 15 (Pipeline de Datos)")
            elif total_images >= 350:
                print(f"\n⚠️ Dataset aceptable pero limitado ({total_images:,} imágenes)")
                print(f"💡 Considera descargar más imágenes o usar data augmentation agresivo")
            else:
                print(f"\n❌ Dataset insuficiente ({total_images:,} imágenes)")
                print(f"💡 Ejecuta el BLOQUE 10 nuevamente para descargar más imágenes")
            
            return df_result
        
        # Ejecutar análisis por raza
        try:
            breed_analysis = analyze_breeds_for_training(df_breed_analysis)
            print(f"\n✅ Análisis por raza completado exitosamente")
        except Exception as e:
            print(f"\n❌ Error en análisis por raza: {e}")
            import traceback
            traceback.print_exc()

print(f"\n{'=' * 60}")


## 🔧 Día 5-6: Preparar Pipeline de Datos


In [ ]:
# ============================================================
# BLOQUE 15: PIPELINE DE DATOS OPTIMIZADO
# ============================================================
# 🔧 Crea pipeline de datos usando módulos del proyecto
# ⚠️ Funciona con CID Dataset (BLOQUE 12) o dataset scrapeado (BLOQUE 10)
# 💡 Usa: CattleDataGenerator (data.data_loader) y get_aggressive_augmentation (data.augmentation)

import pandas as pd
import numpy as np
from pathlib import Path

# Verificar que los módulos están importados (BLOQUE 2)
try:
    from data.data_loader import CattleDataGenerator
    from data.augmentation import get_aggressive_augmentation, get_validation_transform
    print("✅ Módulos del proyecto importados correctamente")
except ImportError as e:
    print(f"❌ Error importando módulos del proyecto: {e}")
    print("💡 Ejecuta el BLOQUE 2 primero para importar los módulos")
    raise

print("=" * 60)
print("🔧 PIPELINE DE DATOS OPTIMIZADO (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Intentar cargar dataset disponible
df_pipeline = None
dataset_name = "Desconocido"
base_data_dir = None

# 1. Intentar usar df_cid si está disponible (BLOQUE 12)
if 'df_cid' in globals() and df_cid is not None:
    df_pipeline = df_cid.copy()
    dataset_name = "CID Dataset"
    base_data_dir = RAW_DIR / 'cid' / 'CID'
    print(f"✅ Usando CID Dataset: {len(df_pipeline):,} registros")
else:
    # 2. Intentar cargar metadata del dataset scrapeado (BLOQUE 10)
    scraped_metadata = RAW_DIR / 'scraped' / 'metadata.csv'
    if scraped_metadata.exists():
        try:
            df_pipeline = pd.read_csv(scraped_metadata)
            dataset_name = "Dataset Scrapeado"
            base_data_dir = RAW_DIR / 'scraped'
            print(f"✅ Usando Dataset Scrapeado: {len(df_pipeline):,} registros")
        except Exception as e:
            print(f"⚠️ Error cargando metadata scrapeado: {e}")
    else:
        print("⚠️ No se encontró ningún dataset disponible")
        print("💡 Ejecuta el BLOQUE 10 para descargar imágenes o el BLOQUE 12 para CID Dataset")

if df_pipeline is None or len(df_pipeline) == 0:
    print("\n❌ No hay datos disponibles para pipeline")
    print("💡 Ejecuta el BLOQUE 10 para descargar imágenes")
    raise ValueError("No hay dataset disponible para crear el pipeline")

# Verificar y ajustar columnas para CattleDataGenerator
# CattleDataGenerator espera: 'image_filename', 'weight_kg', 'breed'
if 'image_path' in df_pipeline.columns:
    # Convertir image_path a image_filename (ruta relativa)
    if base_data_dir:
        def get_relative_path(path_str):
            path = Path(path_str)
            if path.is_absolute():
                try:
                    return path.relative_to(base_data_dir)
                except ValueError:
                    return Path(path.name)  # Si no es relativo, usar solo el nombre
            return path
        df_pipeline['image_filename'] = df_pipeline['image_path'].apply(get_relative_path)
    else:
        df_pipeline['image_filename'] = df_pipeline['image_path'].apply(lambda x: Path(x).name)

# Verificar columnas requeridas
required_cols = ['image_filename', 'weight_kg', 'breed']
missing_cols = [col for col in required_cols if col not in df_pipeline.columns]
if missing_cols:
    print(f"❌ Faltan columnas requeridas: {missing_cols}")
    print(f"💡 Columnas disponibles: {list(df_pipeline.columns)}")
    raise ValueError(f"Columnas requeridas faltantes: {missing_cols}")

print(f"\n✅ Dataset cargado: {len(df_pipeline):,} registros")
print(f"📊 Columnas: {list(df_pipeline.columns)}")
print(f"📁 Directorio base de imágenes: {base_data_dir}")

# Dividir datos en train/val/test
print("\n📊 Dividiendo datos en train/val/test...")
df_shuffled = df_pipeline.sample(frac=1, random_state=42).reset_index(drop=True)

n_total = len(df_shuffled)
n_train = int(n_total * (1 - CONFIG['validation_split'] - CONFIG['test_split']))
n_val = int(n_total * CONFIG['validation_split'])

df_train = df_shuffled[:n_train]
df_val = df_shuffled[n_train:n_train + n_val]
df_test = df_shuffled[n_train + n_val:]

print(f"📈 Train: {len(df_train):,} ({len(df_train)/n_total*100:.1f}%)")
print(f"📈 Val: {len(df_val):,} ({len(df_val)/n_total*100:.1f}%)")
print(f"📈 Test: {len(df_test):,} ({len(df_test)/n_total*100:.1f}%)")

# Crear generadores usando módulos del proyecto
print(f"\n🔧 Creando generadores de datos usando CattleDataGenerator...")

# Augmentation para entrenamiento (agresivo para dataset pequeño)
train_transform = get_aggressive_augmentation(image_size=CONFIG['image_size'])
val_transform = get_validation_transform(image_size=CONFIG['image_size'])

# Crear generadores
train_generator = CattleDataGenerator(
    annotations_df=df_train,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=train_transform,
    shuffle=True
)

val_generator = CattleDataGenerator(
    annotations_df=df_val,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=val_transform,
    shuffle=False
)

test_generator = CattleDataGenerator(
    annotations_df=df_test,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=val_transform,
    shuffle=False
)

print(f"\n✅ BLOQUE 15 COMPLETADO")
print(f"📊 Generadores creados usando módulos del proyecto:")
print(f"   - Train: {len(df_train):,} imágenes ({len(train_generator)} batches)")
print(f"   - Val: {len(df_val):,} imágenes ({len(val_generator)} batches)")
print(f"   - Test: {len(df_test):,} imágenes ({len(test_generator)} batches)")
print(f"💡 Continúa con el BLOQUE 16 para crear la arquitectura del modelo")


In [ ]:
# ============================================================
# BLOQUE 16: ARQUITECTURA DEL MODELO
# ============================================================
# 🏗️ Crea modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 15 ejecutado (generadores creados)
# 💡 Usa: BreedWeightEstimatorCNN.build_generic_model() (models.cnn_architecture)

print("=" * 60)
print("🏗️ ARQUITECTURA DEL MODELO (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que los módulos están importados (BLOQUE 2)
try:
    from models.cnn_architecture import BreedWeightEstimatorCNN
    print("✅ Módulo BreedWeightEstimatorCNN importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Ejecuta el BLOQUE 2 primero para importar los módulos")
    raise

# Crear modelo genérico usando módulo del proyecto
print("🏗️ Creando modelo genérico usando BreedWeightEstimatorCNN...")
print(f"📊 Configuración:")
print(f"   - Image size: {CONFIG['image_size']}")
print(f"   - Base architecture: EfficientNetB1 (desde módulo)")

# Usar build_generic_model del módulo del proyecto
model = BreedWeightEstimatorCNN.build_generic_model(
    input_shape=CONFIG['image_size'] + (3,),
    base_architecture='efficientnetb1'  # O 'mobilenetv2' para más rápido
)

# Re-compilar con learning rate personalizado
model.compile(
    optimizer=optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    loss='mse',
    metrics=['mae', 'mse']
)

print(f"\n✅ Modelo creado con {model.count_params():,} parámetros")
print(f"📊 Arquitectura: {model.name}")
print(f"💡 Usando módulo del proyecto: models.cnn_architecture.BreedWeightEstimatorCNN")

# Mostrar resumen
print(f"\n📐 Resumen del modelo:")
model.summary()


In [ ]:
# ============================================================
# BLOQUE 17: CONFIGURACIÓN DE ENTRENAMIENTO
# ============================================================
# ⚙️ Configura callbacks (EarlyStopping, ReduceLR, ModelCheckpoint, TensorBoard)
# ⚠️ Requiere: BLOQUE 16 ejecutado (modelo creado)

def setup_training_callbacks():
    """Configurar callbacks para entrenamiento"""
    print("⚙️ Configurando callbacks de entrenamiento...")
    
    callbacks_list = [
        # Early stopping
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['early_stopping_patience'],
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce learning rate on plateau
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Model checkpoint
        callbacks.ModelCheckpoint(
            filepath=str(MODELS_DIR / 'best_model.h5'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        
        # TensorBoard
        callbacks.TensorBoard(
            log_dir=str(BASE_DIR / 'logs'),
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    
    print(f"✅ {len(callbacks_list)} callbacks configurados")
    return callbacks_list

# Configurar callbacks
training_callbacks = setup_training_callbacks()

# Configurar MLflow
def start_mlflow_run():
    """Iniciar run de MLflow"""
    # Detectar nombre del dataset usado
    dataset_used = 'Scraped'
    if 'df_cid' in globals() and df_cid is not None:
        dataset_used = 'CID'
    elif 'dataset_name' in globals():
        dataset_used = dataset_name.replace('Dataset', '').strip()
    
    run = mlflow.start_run(run_name=f"cattle-weight-{dataset_used.lower()}-model")

    mlflow.log_params({
        'dataset': dataset_used,
        'model': 'EfficientNetB0',
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate'],
        'epochs': CONFIG['epochs'],
        'image_size': CONFIG['image_size'],
        'augmentation': 'Albumentations'
    })

    print(f"🔬 MLflow run iniciado: {run.info.run_id}")
    print(f"📊 Dataset registrado: {dataset_used}")
    return run

# Iniciar MLflow run
mlflow_run = start_mlflow_run()


In [ ]:
# ============================================================
# BLOQUE 18: ENTRENAMIENTO DEL MODELO
# ============================================================
# 🚀 Entrena el modelo base (puede tardar horas con GPU)
# ⚠️ Requiere: BLOQUE 17 ejecutado (callbacks configurados)
# ⚠️ Tiempo estimado: 2-4 horas con GPU T4 (100 épocas)

def train_model():
    """Entrenar modelo base usando generadores del proyecto"""
    print("🚀 Iniciando entrenamiento del modelo base...")
    print(f"📊 Configuración: {CONFIG}")
    
    # Verificar que los generadores existen
    if 'train_generator' not in globals() or 'val_generator' not in globals():
        raise ValueError("Generadores no encontrados. Ejecuta el BLOQUE 15 primero.")
    
    # Calcular steps por época (usando generadores)
    steps_per_epoch = len(train_generator)
    validation_steps = len(val_generator)
    
    print(f"📈 Steps por época: {steps_per_epoch}")
    print(f"📈 Validation steps: {validation_steps}")
    print(f"💡 Usando generadores del proyecto (CattleDataGenerator)")
    
    # Entrenar modelo usando generadores
    history = model.fit(
        train_generator,
        epochs=CONFIG['epochs'],
        validation_data=val_generator,
        callbacks=training_callbacks,
        verbose=1
    )
    
    print("✅ Entrenamiento completado")
    return history

# Entrenamiento real (requiere generadores preparados y tiempo de ejecución con GPU)
history = train_model()


In [ ]:
# ============================================================
# BLOQUE 19: EVALUACIÓN DEL MODELO
# ============================================================
# 📊 Evalúa el modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 18 ejecutado (modelo entrenado)
# 💡 Usa: MetricsCalculator (models.evaluation.metrics)

import numpy as np

print("=" * 60)
print("📊 EVALUACIÓN DEL MODELO (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que los módulos están importados (BLOQUE 2)
try:
    from models.evaluation.metrics import MetricsCalculator, ModelMetrics
    print("✅ Módulo MetricsCalculator importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Ejecuta el BLOQUE 2 primero para importar los módulos")
    raise

# Verificar que el generador de test existe
if 'test_generator' not in globals():
    raise ValueError("Generador de test no encontrado. Ejecuta el BLOQUE 15 primero.")

def evaluate_model():
    """Evaluar modelo en conjunto de test usando MetricsCalculator"""
    print("📊 Evaluando modelo en conjunto de test...")
    
    # Obtener predicciones y valores reales
    y_true = []
    y_pred = []
    
    print("🔍 Generando predicciones...")
    for i in range(len(test_generator)):
        batch_images, batch_targets = test_generator[i]
        predictions = model.predict(batch_images, verbose=0)
        y_true.extend(batch_targets.flatten())
        y_pred.extend(predictions.flatten())
    
    # Convertir a numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    print(f"📊 Total predicciones: {len(y_true):,}")
    
    # Calcular métricas usando módulo del proyecto
    print("\n📈 Calculando métricas usando MetricsCalculator...")
    metrics = MetricsCalculator.calculate_metrics(
        y_true=y_true,
        y_pred=y_pred,
        breed_type='generic'
    )
    
    # Mostrar resultados
    print(f"\n📈 RESULTADOS DE EVALUACIÓN:")
    print(f"   R²: {metrics.r2_score:.4f}")
    print(f"   MAE: {metrics.mae_kg:.2f} kg")
    print(f"   MSE: {metrics.mse_kg:.2f}")
    print(f"   MAPE: {metrics.mape_percent:.2f}%")
    print(f"   Bias: {metrics.bias_kg:.2f} kg")
    
    # Verificar objetivos (con validación opcional)
    print(f"\n🎯 VERIFICACIÓN DE OBJETIVOS:")
    r2_ok = metrics.r2_score >= CONFIG['target_r2']
    mae_ok = metrics.mae_kg < CONFIG['max_mae']
    
    print(f"   R² ≥ {CONFIG['target_r2']}: {'✅' if r2_ok else '⚠️'} ({metrics.r2_score:.4f})")
    print(f"   MAE < {CONFIG['max_mae']} kg: {'✅' if mae_ok else '⚠️'} ({metrics.mae_kg:.2f} kg)")
    
    if not (r2_ok and mae_ok):
        print(f"\n💡 NOTA: Los objetivos no se cumplieron completamente")
        print(f"💡 Esto es normal en un primer entrenamiento. Puedes:")
        print(f"   - Ajustar hiperparámetros")
        print(f"   - Entrenar más épocas")
        print(f"   - Usar fine-tuning")
    
    # Log métricas en MLflow
    if 'mlflow' in globals() and ('mlflow_available' in globals() and mlflow_available):
        mlflow.log_metrics({
            'test_r2': metrics.r2_score,
            'test_mae_kg': metrics.mae_kg,
            'test_mse_kg': metrics.mse_kg,
            'test_mape_percent': metrics.mape_percent,
            'test_bias_kg': metrics.bias_kg
        })
    
    return metrics.to_dict()

# Evaluar modelo
evaluation_results = evaluate_model()

print(f"\n✅ BLOQUE 19 COMPLETADO")
print(f"💡 Métricas calculadas usando módulo del proyecto: MetricsCalculator")


In [ ]:
# ============================================================
# BLOQUE 20: EXPORTAR A TFLITE
# ============================================================
# 📱 Exporta modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 19 ejecutado (modelo evaluado)
# 💡 Usa: TFLiteExporter (models.export.tflite_converter)

print("=" * 60)
print("📱 EXPORTAR A TFLITE (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que los módulos están importados (BLOQUE 2)
try:
    from models.export.tflite_converter import TFLiteExporter
    print("✅ Módulo TFLiteExporter importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Ejecuta el BLOQUE 2 primero para importar los módulos")
    raise

# Guardar modelo temporalmente para conversión
print("💾 Guardando modelo temporalmente para conversión...")
temp_model_path = MODELS_DIR / 'temp_model.h5'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
model.save(str(temp_model_path))
print(f"✅ Modelo guardado en: {temp_model_path}")

# Exportar usando TFLiteExporter del proyecto
tflite_path = MODELS_DIR / 'generic-cattle-v1.0.0.tflite'

print(f"\n📱 Exportando modelo a TFLite usando TFLiteExporter...")
print(f"📁 Archivo de salida: {tflite_path}")

# Usar TFLiteExporter del proyecto (optimización FP16 por defecto)
model_size_bytes = TFLiteExporter.convert_to_tflite(
    saved_model_path=str(temp_model_path),
    output_path=str(tflite_path),
    optimization='default'  # FP16: reduce 2x el tamaño, mantiene precisión
)

model_size_kb = model_size_bytes / 1024
model_size_mb = model_size_kb / 1024

# Log en MLflow
if 'mlflow' in globals() and ('mlflow_available' in globals() and mlflow_available):
    mlflow.log_artifact(str(tflite_path))
    mlflow.log_metric('model_size_kb', model_size_kb)
    mlflow.log_metric('model_size_mb', model_size_mb)

# Limpiar modelo temporal
try:
    temp_model_path.unlink()
    print(f"🧹 Modelo temporal eliminado")
except:
    pass

print(f"\n✅ BLOQUE 20 COMPLETADO")
print(f"🎯 MODELO BASE LISTO PARA INTEGRACIÓN")
print(f"📁 Archivo: {tflite_path}")
print(f"📏 Tamaño: {model_size_mb:.2f} MB ({model_size_kb:.1f} KB)")
print(f"💡 Usando módulo del proyecto: TFLiteExporter")
if 'mlflow_run' in globals():
    print(f"🔬 MLflow run: {mlflow_run.info.run_id}")


## 📋 Resumen y Próximos Pasos


In [ ]:
# ============================================================
# BLOQUE 21: RESUMEN FINAL
# ============================================================
# 📋 Genera resumen completo del trabajo realizado
# ⚠️ Requiere: Todos los bloques anteriores ejecutados
# 💾 Guarda resumen en DATA_DIR/final_summary.json

import json
import pandas as pd
from pathlib import Path

def generate_final_summary():
    """Generar resumen final del trabajo realizado"""
    print("📋 RESUMEN FINAL - PERSONA 2: SETUP ML")
    print("=" * 60)
    
    # Resumen de datasets
    print(f"\n📥 DATASETS PROCESADOS:")
    
    # CID Dataset
    cid_images = 0
    if 'df_cid' in globals() and df_cid is not None:
        cid_images = len(df_cid)
        print(f"   ✅ CID Dataset: {cid_images:,} imágenes")
    elif 'datasets_summary' in globals():
        try:
            cid_row = datasets_summary[datasets_summary['name'] == 'CID Dataset']
            cid_images = int(cid_row['images'].iloc[0]) if not cid_row.empty else 0
            print(f"   {'✅' if cid_images else '⚠️'} CID Dataset: {cid_images:,} imágenes")
        except:
            print(f"   ⚠️ CID Dataset: No disponible")
    else:
        print(f"   ⚠️ CID Dataset: No disponible")
    
    # Scraped Dataset
    scraped_count = 0
    if 'scraped_images' in globals():
        scraped_count = scraped_images
        print(f"   ✅ Google Images Scraped: {scraped_count:,} imágenes")
    elif 'df_pipeline' in globals():
        scraped_count = len(df_pipeline)
        print(f"   ✅ Dataset Scrapeado: {scraped_count:,} imágenes")
    elif 'datasets_summary' in globals():
        try:
            scraped_row = datasets_summary[datasets_summary['name'] == 'Google Images Scraped']
            scraped_count = int(scraped_row['images'].iloc[0]) if not scraped_row.empty else 0
            print(f"   {'✅' if scraped_count else '⚠️'} Google Images Scraped: {scraped_count:,} imágenes")
        except:
            print(f"   ⚠️ Google Images Scraped: No disponible")
    else:
        print(f"   ⚠️ Google Images Scraped: No disponible")
    
    # Kaggle Dataset
    if 'kaggle_dataset_path' in globals() and kaggle_dataset_path and Path(kaggle_dataset_path).exists():
        kaggle_images = len(list(Path(kaggle_dataset_path).glob('**/*.jpg')))
        kaggle_id = KAGGLE_DATASET_ID if 'KAGGLE_DATASET_ID' in globals() else 'N/A'
        status_icon = '✅' if kaggle_images else '⚠️'
        print(f"   {status_icon} Kaggle Dataset ({kaggle_id}): {kaggle_images:,} imágenes")
    else:
        print("   ⚠️ Kaggle Dataset: Pendiente configuración (opcional)")
    
    # Resumen de análisis
    print(f"\n📊 ANÁLISIS COMPLETADO:")
    print(f"   ✅ EDA completo con visualizaciones")
    print(f"   ✅ Análisis por raza para estrategia de entrenamiento")
    print(f"   ✅ Pipeline de datos optimizado")
    
    # Resumen de modelo
    print(f"\n🤖 MODELO BASE:")
    if 'model' in globals():
        print(f"   ✅ Arquitectura: {model.name if hasattr(model, 'name') else 'EfficientNetB1'}")
        print(f"   ✅ Parámetros: {model.count_params():,}")
    else:
        print(f"   ⚠️ Modelo: No disponible")
    
    if 'model_size_kb' in globals():
        print(f"   ✅ TFLite exportado: {model_size_kb / 1024:.2f} MB ({model_size_kb:.1f} KB)")
    elif 'model_size' in globals():
        print(f"   ✅ TFLite exportado: {model_size / 1024:.2f} MB ({model_size:.1f} KB)")
    else:
        print(f"   ⚠️ TFLite: No exportado aún")
    
    if 'mlflow_run' in globals():
        print(f"   ✅ MLflow tracking: {mlflow_run.info.run_id}")
    else:
        print(f"   ⚠️ MLflow: No disponible")
    
    # Próximos pasos
    print(f"\n🎯 PRÓXIMOS PASOS:")
    print(f"   1. 🔄 Fine-tuning por raza (Semanas 3-6)")
    print(f"   2. 📸 Recolección Criollo + Pardo Suizo (Semanas 7-8)")
    print(f"   3. 🧪 Entrenamiento final (Semanas 9-10)")
    print(f"   4. 📱 Integración en app móvil")
    
    # Guardar resumen
    total_images = cid_images + scraped_count
    datasets_count = sum([1 if cid_images > 0 else 0, 1 if scraped_count > 0 else 0])
    
    # Obtener tamaño del modelo
    model_size_value = 0
    if 'model_size_kb' in globals():
        model_size_value = model_size_kb
    elif 'model_size' in globals():
        model_size_value = model_size
    
    # Obtener arquitectura del modelo
    model_arch = 'EfficientNetB1'
    if 'model' in globals() and hasattr(model, 'name'):
        model_arch = model.name
    
    summary_data = {
        'completion_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'datasets_processed': datasets_count,
        'total_images': total_images,
        'cid_images': cid_images,
        'scraped_images': scraped_count,
        'model_architecture': model_arch,
        'model_size_kb': model_size_value,
        'mlflow_run_id': mlflow_run.info.run_id if 'mlflow_run' in globals() else 'N/A',
        'status': 'COMPLETADO'
    }
    
    if 'DATA_DIR' in globals():
        summary_path = DATA_DIR / 'final_summary.json'
    else:
        summary_path = RAW_DIR.parent / 'processed' / 'final_summary.json'
        summary_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=2)

    if 'mlflow_run' in globals():
        mlflow.end_run()
    
    print(f"\n💾 Resumen guardado en: {summary_path}")
    print(f"\n🎉 PERSONA 2: SETUP ML COMPLETADO EXITOSAMENTE")

# Generar resumen final
generate_final_summary()


## 📝 Notas Importantes

### ⚠️ Configuración Requerida
1. **Kaggle API**: Subir `kaggle.json` para descargar datasets
2. **CID Dataset**: Reemplazar URL simulada con URL real
3. **CattleEyeView**: Solicitar acceso a autores del paper

### 🔧 Optimizaciones Implementadas
- **Mixed Precision**: FP16 para acelerar entrenamiento
- **Data Pipeline**: Cache + prefetch + shuffle optimizado
- **Augmentation**: Albumentations específico para ganado
- **TFLite Export**: Optimizado para móvil

### 📊 Métricas Objetivo
- **R² ≥ 0.95**: Explicación 95% de varianza
- **MAE < 5 kg**: Error absoluto promedio
- **Inference < 3s**: Tiempo en móvil

### 🎯 Estado Actual
- ✅ **Infraestructura ML**: Completada
- ✅ **Pipeline de datos**: Optimizado
- ✅ **Modelo base**: Listo para fine-tuning
- 🔄 **Próximo**: Fine-tuning por raza específica
